# 유저 행동 이벤트 로그 생성

In [1]:
# ===================================================================
# 1️⃣ 필수 라이브러리 Import
# ===================================================================

import os
import uuid
import json
import random
import gc
from datetime import datetime, timedelta, timezone, date
from typing import List, Dict, Any, Optional
import warnings
import pytz

# 데이터 처리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any, Optional

from datetime import datetime
import time
from dotenv import load_dotenv
import gc
import tqdm

from faker import Faker

import dask
import dask.dataframe as dd
import dask.bag as db
from dask.diagnostics import ProgressBar
from dask import delayed

# 시각화 (필요시)
import matplotlib.pyplot as plt
import seaborn as sns

import pytz

from elasticsearch import Elasticsearch, helpers
from elasticsearch.exceptions import ConnectionError, ConnectionTimeout

plt.rcParams['font.family'] = 'Nanum Gothic'
plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다.")

✅ 모든 라이브러리가 성공적으로 로드되었습니다.


## 1. DB 데이터 불러오기

In [2]:
# ===================================================================
# 2️⃣ 데이터 로딩 및 기본 설정
# ===================================================================

# 레시피 데이터 읽기
recipes_df = pd.read_parquet('data/output/total_recipes.parquet')

# 사용자 데이터 읽기
users_df = pd.read_parquet('data/output/user.parquet')

# 사용자 프로필 데이터 읽기
profiles_df = pd.read_parquet('data/output/user_profiles.parquet')

print(f"✅ 데이터 로딩 완료:")
print(f"   - 레시피: {len(recipes_df):,}개")
print(f"   - 사용자: {len(users_df):,}명")
print(f"   - 프로필: {len(profiles_df):,}개")

# Demographic Segment 분포 정의
DEMOGRAPHIC_DISTRIBUTION = {
    'FEMALE_20S': 0.142,    # 14.2%
    'FEMALE_30S': 0.207,    # 20.7%
    'FEMALE_40_PLUS': 0.356, # 35.6%
    'MALE_20S': 0.062,      # 6.2%
    'MALE_30S': 0.085,      # 8.5%
    'MALE_40_PLUS': 0.148   # 14.8%
}

# 행동 태그 정의
INTENSITY_PERSONAS = {
    'POWER_USER': {'ratio': 0.15, 'description': '파워_유저, 주 5회 이상 활동'},
    'ACTIVE_USER': {'ratio': 0.55, 'description': '활성_유저, 주 2-4회 활동'},
    'CASUAL_USER': {'ratio': 0.30, 'description': '캐주얼_유저, 주 1회 이하 활동'}
}

COOKING_STYLE_PERSONAS = {
    'DESSERT_FOCUSED': {'ratio': 0.20, 'description': '디저트_중심, 베이킹 디저트 제작 선호'},
    'HEALTHY_CONSCIOUS': {'ratio': 0.25, 'description': '건강식_지향, 다이어트 웰빙 요리 선호'},
    'COMFORT_FOOD': {'ratio': 0.25, 'description': '든든한_식사, 메인 요리 한 끼 식사 선호'},
    'QUICK_CONVENIENT': {'ratio': 0.20, 'description': '간편_요리, 시간절약 간단 요리 선호'},
    'DIVERSE_EXPLORER': {'ratio': 0.10, 'description': '다양한_탐험, 특별한 패턴 없이 다양하게 탐색'}
}

In [3]:
category_ranking_dict = {
    "상의": {
        "맨투맨/스웨트": "001005",
        "후드 티셔츠": "001004",
        "셔츠/블라우스": "001002",
        "긴소매 티셔츠": "001010",
        "반소매 티셔츠": "001001",
        "피케/카라 티셔츠": "001003",
        "니트/스웨터": "001006",
        "민소매 티셔츠": "001011",
        "기타 상의": "001008"
    },
    'click_auth_button': {
        'required_props': ['type'],
        'optional_props': [],
        'next_events': ['auth_success']
    },
    'auth_success': {
        'required_props': ['method', 'type'],
        'optional_props': [],
        'next_events': ['view_page', 'search_recipe']
    },
    'search_recipe': {
        'required_props': ['search_type'],
        'optional_props': ['search_keyword', 'selected_filters', 'result_count'],
        'next_events': ['view_recipe_list', 'click_recipe_from_list']
    },
    'view_recipe_list': {
        'required_props': ['list_type'],
        'optional_props': ['displayed_recipe_ids'],
        'next_events': ['click_recipe_from_list', 'search_recipe']
    },
    'click_recipe_from_list': {
        'required_props': ['recipe_id'],
        'optional_props': ['rank'],
        'next_events': ['click_bookmark', 'click_like', 'create_comment']
    },
    'click_bookmark': {
        'required_props': ['recipe_id', 'action'],
        'optional_props': [],
        'next_events': ['view_page', 'click_recipe_from_list']
    },
    'click_like': {
        'required_props': ['recipe_id', 'action'],
        'optional_props': [],
        'next_events': ['view_page', 'click_recipe_from_list', 'create_comment']
    },
    'create_comment': {
        'required_props': ['recipe_id'],
        'optional_props': ['comment_length'],
        'next_events': ['view_page', 'click_recipe_from_list']
    },
    'create_recipe_success': {
        'required_props': ['recipe_id'],
        'optional_props': ['category', 'ingredient_count'],
        'next_events': ['view_page']
    },
    'view_ads': {
        'required_props': ['ad_id'],
        'optional_props': ['ad_type', 'position'],
        'next_events': ['click_ads', 'view_page']
    },
    'click_ads': {
        'required_props': ['ad_id'],
        'optional_props': ['ad_type', 'position', 'target_url'],
        'next_events': ['view_page']
    }
}

In [4]:
EVENT_TYPES = [
    "view_main",
    "search",
    "promotion_view",
    "promotion_click",
    "product_view",
    "add_to_cart",
    "change_cart_item_quantity",
    "remove_from_cart",
    "order",
    "refund",
    "product_like",
    "brand_like",
]

In [5]:
EVENT_START_DATE = datetime.fromisoformat("2023-07-01T12:00:00").replace(tzinfo=timezone.utc)
EVENT_END_DATE = datetime.fromisoformat("2025-06-30T12:00:00").replace(tzinfo=timezone.utc)

In [6]:
# 2025년 6월 16일 오후 2시 48분 16초 KST 기준, 최종 '행동 빈도 기반' 동적 시나리오
EVENT_CONFIG = {
    # ===================================================================
    # I. 시나리오 기본 설정 (변경 없음)
    # ===================================================================
    "SCENARIO_INFO": {
        "description": "행동 빈도 및 전환율 기반 5개 세그먼트 동적 성장 모델 (2년)",
        "total_duration_days": 730,
        "total_log_target": 11_600_000,
        "start_dau": 100, "end_dau": 750,
        "final_total_users": 50_000,
    },

    # ===================================================================
    # II. 세그먼트별 행동 정의 (가장 큰 변화)
    # ===================================================================
    "SEGMENT_BEHAVIORS": {
        "Bouncer": {
            "avg_events": 5,
            "action_probabilities": { 
                "search": 0.05, 
                "promotion_view": 0.1, 
                "product_view": 0.2, 
                "view_main": 0.9,  # 추가: 바운서는 메인 페이지를 많이 보지만 깊이 들어가지 않음
                "like": 0, 
                "add_to_cart": 0, 
                "order": 0,
                "promotion_click_rate": 0.01
            },
            "action_counts": { 
                "max_product_views": 3,
                "max_view_main": 3  # 추가: 메인 페이지 최대 방문 횟수
            }
        },
        "Casual_Browser": {
            "avg_events": 20,
            "action_probabilities": { 
                "search": 0.6, 
                "promotion_view": 0.8, 
                "product_view": 0.9, 
                "view_main": 0.8,  # 추가: 캐주얼 브라우저도 메인 페이지 자주 방문
                "like": 0.1, 
                "add_to_cart": 0.01, 
                "order": 0,
                "promotion_click_rate": 0.05
            },
            "action_counts": { 
                "max_searches": 2, 
                "max_promotion_views": 10, 
                "max_product_views": 20, 
                "max_likes": 2,
                "max_view_main": 4  # 추가
            }
        },
        "Cart_User": {
            "avg_events": 50,
            "action_probabilities": { 
                "search": 0.8, 
                "promotion_view": 0.9, 
                "product_view": 0.95, 
                "view_main": 0.7,  # 추가: 목적이 있어서 메인 페이지 방문 빈도는 상대적으로 낮음
                "like": 0.4, 
                "add_to_cart": 0.7, 
                "order": 0,
                "promotion_click_rate": 0.15
            },
            "action_counts": { 
                "max_searches": 4, 
                "max_promotion_views": 20, 
                "max_product_views": 40, 
                "max_likes": 15, 
                "max_cart_additions": 3, 
                "max_cart_item_charges": 2, 
                "max_cart_item_removals": 1,
                "max_view_main": 3  # 추가
            }
        },
        "Purchaser": {
            "avg_events": 70,
            "action_probabilities": { 
                "search": 0.8, 
                "promotion_view": 0.9, 
                "product_view": 0.95, 
                "view_main": 0.6,  # 추가: 구매 목적이 명확해서 메인 페이지 덜 방문
                "like": 0.5, 
                "add_to_cart": 0.8, 
                "order": 0.25,
                "promotion_click_rate": 0.20
            },
            "action_counts": { 
                "max_searches": 5, 
                "max_promotion_views": 20, 
                "max_product_views": 50, 
                "max_likes": 20, 
                "max_cart_additions": 4, 
                "max_cart_item_charges": 3, 
                "max_cart_item_removals": 2,
                "max_view_main": 2  # 추가
            }
        },
        "Loyal_Customer": {
            "avg_events": 130,
            "action_probabilities": { 
                "search": 0.9, 
                "promotion_view": 0.95, 
                "product_view": 0.98, 
                "view_main": 0.85,  # 추가: 충성 고객은 새로운 상품 확인을 위해 메인 페이지 자주 방문
                "like": 0.8, 
                "add_to_cart": 0.9, 
                "order": 0.4, 
                "refund": 0.05,
                "promotion_click_rate": 0.25
            },
            "action_counts": { 
                "max_searches": 6, 
                "max_promotion_views": 25, 
                "max_product_views": 80, 
                "max_likes": 30, 
                "max_cart_additions": 5, 
                "max_cart_item_charges": 4, 
                "max_cart_item_removals": 3,
                "max_view_main": 5  # 추가: 충성 고객은 메인 페이지를 가장 많이 방문
            }
        }
    },

    # ===================================================================
    # III. 성장 단계별 동적 설정 (구조는 유지, 내부 로직은 확률 기반으로 해석)
    # ===================================================================
    # 각 성장 단계별로 세그먼트 분포와 익명 사용자 전환율을 정의합니다.
    "PHASE_SETTINGS": {
        # 초기 단계: 사용자 기반이 적고, 익명 사용자의 전환율이 높음
        "EARLY": {
            "duration_months": 6,
            "segment_distribution": { "Bouncer": 0.10, "Casual_Browser": 0.40, "Cart_User": 0.15, "Purchaser": 0.15, "Loyal_Customer": 0.20 },
            "anonymous_to_member_conversion_rate": 0.20
        },
        # 성장 단계: 사용자 기반이 증가하고, 익명 사용자의 전환율이 감소함
        "GROWTH": {
            "duration_months": 12,
            "segment_distribution": { "Bouncer": 0.25, "Casual_Browser": 0.40, "Cart_User": 0.25, "Purchaser": 0.08, "Loyal_Customer": 0.02 },
            "anonymous_to_member_conversion_rate": 0.10
        },
        # 성숙 단계: 사용자 기반이 안정되고, 익명 사용자의 전환율이 더욱 감소함
        "MATURE": {
            "duration_months": 6,
            "segment_distribution": { "Bouncer": 0.15, "Casual_Browser": 0.35, "Cart_User": 0.25, "Purchaser": 0.15, "Loyal_Customer": 0.10 },
            "anonymous_to_member_conversion_rate": 0.15
        }
    }
}

In [7]:
# EVENT_CONFIG = {
#     # 퍼널 전환율
#     "search_to_product_view_rate": 0.5,      # 검색 후 제품 조회로 이어지는 확률 (Search → Product View conversion rate)
#     # "view_to_cart_rate": 0.1,              # 제품 조회 후 장바구니 추가 확률 (Product View → Add to Cart rate)
#     "view_to_order_rate": 0.03,              # 제품 조회 후 주문으로 이어지는 확률 (Product View → Order rate)
#     "view_to_like_rate": 0.2,               # 로그인 사용자 중 제품 조회 후 좋아요로 이어지는 확률 (Product View → Like rate)
#     # "anonymous_to_login_rate": 0.3,         # 익명 사용자 중 로그인 (회원) 전환율 (Anonymous → Login conversion rate)
#     "cart_to_order_rate": 0.3,              # 장바구니 담기 후 주문으로 이어지는 확률 (Add to Cart → Order rate)
#     "order_to_cancel_rate": 0.02,           # 주문 후 취소로 이어지는 확률 (Order → Cancel rate)

#     # 사용자 세션 설정
#     # "anonymous_session_rate": 0.6,          # 전체 세션 중 익명 세션 비율 (Anonymous session rate)
#     "session_per_records_ratio": 5,         # 레코드 수 대비 세션 수 비율(1/X) (Sessions per Records ratio)
#     "max_sessions": 100,                    # 생성 가능한 최대 세션 수 (Maximum number of sessions)
#     "anonymous_ids_per_records_ratio": 80,  # 레코드 수 대비 익명 ID 수 비율(1/X) (Anonymous IDs per Records ratio)
#     # "max_anonymous_ids": 500,               # 생성 가능한 최대 익명 ID 수 (Maximum number of anonymous IDs)
#     "reuse_session_rate": 0.7,              # 기존 세션을 재사용할 확률 (Reuse session rate)

#     # 이벤트 생성 관련
#     "main_page_visit_rate": 0.3,            # 세션 시작 시 메인 페이지 방문 확률 (Main page visit rate)
#     "min_searches_per_session": 1,          # 세션당 최소 검색 이벤트 수 (Minimum searches per session)
#     "max_searches_per_session": 5,          # 세션당 최대 검색 이벤트 수 (Maximum searches per session)
#     "min_product_views": 1,                 # 세션당 최소 제품 조회 수 (Minimum product views per session)
#     "max_product_views": 8,                 # 세션당 최대 제품 조회 수 (Maximum product views per session)
#     "min_cart_additions": 1,                # 세션당 최소 장바구니 추가 수 (Minimum cart additions per session)
#     "max_cart_additions": 3,                # 세션당 최대 장바구니 추가 수 (Maximum cart additions per session)

#     # 검색 쿼리 비율
#     "main_category_search_rate": 0.3,       # 메인 카테고리 검색 비율 (Main category search rate)
#     "sub_category_search_rate": 0.4,        # 서브 카테고리 검색 비율 (Subcategory search rate)

#     # 세션 시작 시간 설정
#     "session_start_time_fraction": 0.8,     # 세션 시작 시간 분포 비율 (Session start time fraction)
#     "supplementary_session_time_fraction": 0.9,  # 추가 세션 생성 시 시간 분포 비율 (Supplementary session time fraction)

#     # 이벤트 간격 설정 (초 단위)
#     "main_page_time_min": 2,                # 메인 페이지 방문까지 경과 최소 시간 (Main page time min)
#     "main_page_time_max": 5,                # 메인 페이지 방문까지 경과 최대 시간 (Main page time max)
#     "search_time_min": 4,                   # 검색 이벤트 간 최소 간격 (Search time min)
#     "search_time_max": 180,                 # 검색 이벤트 간 최대 간격 (Search time max)
#     "view_time_min": 5,                     # 제품 조회 이벤트 간 최소 간격 (View time min)
#     "view_time_max": 60,                    # 제품 조회 이벤트 간 최대 간격 (View time max)
#     "like_time_min": 4,                     # 좋아요 이벤트 간 최소 간격 (Like time min)
#     "like_time_max": 30,                    # 좋아요 이벤트 간 최대 간격 (Like time max)
#     "login_time_min": 10,                   # 로그인 이벤트까지 최소 대기 시간 (Login time min)
#     "login_time_max": 60,                   # 로그인 이벤트까지 최대 대기 시간 (Login time max)
#     "redirect_time_min": 1,                 # 페이지 리디렉션 간 최소 시간 (Redirect time min)
#     "redirect_time_max": 5,                 # 페이지 리디렉션 간 최대 시간 (Redirect time max)
#     "cart_time_min": 10,                     # 장바구니 추가까지 최소 대기 시간 (Cart time min)
#     "cart_time_max": 45,                    # 장바구니 추가까지 최대 대기 시간 (Cart time max)
#     "order_time_min": 30,                   # 주문까지 최소 대기 시간 (Order time min)
#     "order_time_max": 300,                   # 주문까지 최대 대기 시간 (Order time max)



#     "view_to_cart_rate": 0.2,              # 0.1에서 0.2로 증가
#     "anonymous_session_rate": 0.3,          # 0.6에서 0.3으로 감소
#     "anonymous_to_login_rate": 0.5,         # 0.3에서 0.5로 증가
# }

## 3. 로그 데이터 생성

In [8]:
# 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning, module='distributed.client')

class OptimizedEventLogGenerator:
    """EVENT_CONFIG 기반 최적화된 이벤트 로그 생성기 (모든 카테고리 보장)"""
    
    def __init__(self, config: Dict, product_df: pd.DataFrame, user_df: pd.DataFrame, 
                 brand_df: pd.DataFrame = None, promotion_df: pd.DataFrame = None,
                 category_ranking_dict: Dict = None,
                 event_start_date: str = "2023-07-01", event_end_date: str = "2025-06-30",
                 target_log_count: int = None):
        self.config = config
        
        # 큰 객체들을 직렬화 가능한 형태로 변환
        self.product_data = self._serialize_dataframe(product_df)
        self.user_data = self._serialize_dataframe(user_df)
        self.brand_data = self._serialize_dataframe(brand_df) if brand_df is not None else None
        self.promotion_data = self._serialize_dataframe(promotion_df) if promotion_df is not None else None
        self.category_ranking_dict = category_ranking_dict or {}
        
        # 날짜 설정
        self.event_start_date = datetime.strptime(event_start_date, "%Y-%m-%d").date()
        self.event_end_date = datetime.strptime(event_end_date, "%Y-%m-%d").date()
        self.total_duration_days = (self.event_end_date - self.event_start_date).days + 1
        
        # 목표 로그 수 설정
        self.target_log_count = target_log_count or config["SCENARIO_INFO"].get("total_log_target", 11600000)
        
        # 사전 계산된 데이터 (직렬화 가능)
        self.product_weights = self._calculate_product_weights()
        self.product_details = self._prepare_product_details_with_guaranteed_coverage()
        self.seasonal_weights = self._initialize_seasonal_weights()
        self.seasonal_queries = self._initialize_seasonal_queries()
        
        # 리스트로 변환하여 직렬화 최적화
        self.user_ids = list(self.user_data.get('user_id', self.user_data.get('id', [])))
        
        print("="*60)
        print("이벤트 로그 생성기 초기화 완료")
        print("="*60)
        print(f"- 제품 수: {len(self.product_data.get('id', []))}개")
        print(f"- 사용자 수: {len(self.user_ids)}개")
        print(f"- 브랜드 수: {len(self.brand_data.get('brand_id', [])) if self.brand_data else 0}개")
        print(f"- 프로모션 수: {len(self.promotion_data.get('brand_id', [])) if self.promotion_data else 0}개")
        print(f"- 시작 날짜: {self.event_start_date}")
        print(f"- 종료 날짜: {self.event_end_date}")
        print(f"- 총 기간: {self.total_duration_days}일")
        print(f"- 목표 로그 수: {self.target_log_count:,}개")
        print("="*60)
    
    def _serialize_dataframe(self, df: pd.DataFrame) -> Dict:
        """DataFrame을 직렬화 가능한 딕셔너리로 변환"""
        if df is None:
            return {}
        
        # 각 컬럼을 리스트로 변환
        result = {}
        for column in df.columns:
            result[column] = df[column].tolist()
        
        return result
    
    def _get_seasonal_main_category_distribution(self, season: str) -> Dict[str, float]:
        """계절별 메인 카테고리 분포 비율 반환"""
        distributions = {
        "spring": {"상의": 0.40, "바지": 0.30, "아우터": 0.25, "신발": 0.05},
        "summer": {"상의": 0.55, "바지": 0.30, "아우터": 0.05, "신발": 0.10},
        "autumn": {"아우터": 0.40, "상의": 0.38, "바지": 0.15, "신발": 0.07},
        "winter": {"아우터": 0.50, "상의": 0.35, "바지": 0.12, "신발": 0.03}
        }
        return distributions.get(season, {"상의": 0.35, "바지": 0.25, "아우터": 0.25, "신발": 0.15})
    
    def _prepare_product_details_with_guaranteed_coverage(self) -> Dict[str, Dict]:
        """모든 카테고리 코드가 최소 1개 이상 포함되도록 보장하는 제품 상세 정보 준비"""
        details = {}
        
        product_ids = self.product_data.get('product_id', self.product_data.get('id', []))
        categories = self.product_data.get('category_code', self.product_data.get('category_id', []))
        brand_ids = self.product_data.get('brand_id', self.product_data.get('brand', []))
        prices = self.product_data.get('price', self.product_data.get('price_amount', []))
        discounted_prices = self.product_data.get('discounted_price', [])
        genders = self.product_data.get('gender', self.product_data.get('target_gender', []))
        product_names = self.product_data.get('product_name', self.product_data.get('name', []))
        
        # 실제 데이터에서 사용 가능한 성별 값들 추출
        unique_genders = set()
        for gender in genders:
            if gender:  # None이 아닌 경우만
                unique_genders.add(gender)
        
        # 기본값 설정 (데이터에 따라 조정)
        available_genders = list(unique_genders) if unique_genders else ["M", "F", "U"]
        print(f"사용 가능한 성별 값: {available_genders}")
        
        # 모든 카테고리 코드 수집
        all_category_codes = []
        category_to_main = {}
        category_to_sub = {}
        
        for main_category, sub_categories in self.category_ranking_dict.items():
            for sub_category, code in sub_categories.items():
                all_category_codes.append(code)
                category_to_main[code] = main_category
                category_to_sub[code] = sub_category
        
        print(f"전체 카테고리 코드 수: {len(all_category_codes)}")
        
        # ---- 1단계: 모든 카테고리 코드를 최소 1개씩 강제 할당 ----
        guaranteed_products = {}
        season_order = ["spring", "summer", "autumn", "winter"]
        
        for i, category_code in enumerate(all_category_codes):
            # 각 카테고리마다 최소 1개 제품 보장
            season_index = i % 4
            preferred_season = season_order[season_index]
            
            # 기존 제품 ID가 있으면 사용, 없으면 새로 생성
            if i < len(product_ids):
                product_id = product_ids[i]
                original_gender = genders[i] if i < len(genders) else random.choice(available_genders)
                original_brand_id = brand_ids[i] if i < len(brand_ids) else str(uuid.uuid4())
                original_product_name = product_names[i] if i < len(product_names) else f"Product_{product_id}"
            else:
                product_id = f"guaranteed_product_{i}"
                original_gender = random.choice(available_genders)
                original_brand_id = str(uuid.uuid4())
                original_product_name = f"Product_{product_id}"
            
            base_price = (prices[i] if i < len(prices) and prices[i] 
                         else random.randint(10, 200))
            discounted_price = (discounted_prices[i] if i < len(discounted_prices) and discounted_prices[i] 
                              else base_price * (100 - random.randint(0, 50)) // 100)
            
            guaranteed_products[product_id] = {
                'id': product_id,
                'category_code': category_code,
                'brand_id': original_brand_id,
                'price': int(base_price),
                'discounted_price': int(discounted_price),
                'gender': original_gender,
                'product_name': original_product_name,
                'preferred_season': preferred_season,
                'is_guaranteed': True  # 보장된 제품 표시
            }
        
        print(f"-- 모든 카테고리 보장: {len(guaranteed_products)}개 제품 생성")
        
        # ---- 2단계: 나머지 제품들을 계절별 분포에 따라 할당 ----
        remaining_product_ids = product_ids[len(all_category_codes):]
        
        for i, pid in enumerate(remaining_product_ids):
            try:
                # 계절 순환 할당
                season_index = i % 4
                current_season = season_order[season_index]
                
                # 기존 카테고리가 유효하면 사용, 아니면 계절별 분포에 따라 할당
                if (i + len(all_category_codes)) < len(categories) and categories[i + len(all_category_codes)] and categories[i + len(all_category_codes)] in all_category_codes:
                    assigned_category = categories[i + len(all_category_codes)]
                else:
                    # 계절별 메인 카테고리 분포에 따라 선택
                    seasonal_dist = self._get_seasonal_main_category_distribution(current_season)
                    selected_main = random.choices(
                        list(seasonal_dist.keys()), 
                        weights=list(seasonal_dist.values())
                    )[0]
                    
                    # 해당 메인 카테고리의 서브 카테고리 중 랜덤 선택
                    available_codes = [code for code, main_cat in category_to_main.items() 
                                     if main_cat == selected_main]
                    assigned_category = random.choice(available_codes)
                
                base_price = (prices[i + len(all_category_codes)] if (i + len(all_category_codes)) < len(prices) and prices[i + len(all_category_codes)] 
                             else random.randint(10, 200))
                discounted_price = (discounted_prices[i + len(all_category_codes)] if (i + len(all_category_codes)) < len(discounted_prices) and discounted_prices[i + len(all_category_codes)] 
                                  else base_price * (100 - random.randint(0, 50)) // 100)
                
                # 기존 제품의 속성들 유지
                original_gender = (genders[i + len(all_category_codes)] if (i + len(all_category_codes)) < len(genders) and genders[i + len(all_category_codes)] 
                                 else random.choice(available_genders))
                original_brand_id = (brand_ids[i + len(all_category_codes)] if (i + len(all_category_codes)) < len(brand_ids) and brand_ids[i + len(all_category_codes)] 
                                   else str(uuid.uuid4()))
                original_product_name = (product_names[i + len(all_category_codes)] if (i + len(all_category_codes)) < len(product_names) and product_names[i + len(all_category_codes)] 
                                       else f"Product_{pid}")
                
                guaranteed_products[pid] = {
                    'id': pid,
                    'category_code': assigned_category,
                    'brand_id': original_brand_id,
                    'price': int(base_price),
                    'discounted_price': int(discounted_price),
                    'gender': original_gender,
                    'product_name': original_product_name,
                    'preferred_season': current_season,
                    'is_guaranteed': False
                }
            except Exception as e:
                print(f"제품 {pid} 처리 중 오류: {e}")
                # 기본값 설정
                guaranteed_products[pid] = {
                    'id': pid,
                    'category_code': random.choice(all_category_codes),
                    'brand_id': str(uuid.uuid4()),
                    'price': random.randint(10, 200),
                    'discounted_price': random.randint(8, 160),
                    'gender': random.choice(available_genders),
                    'product_name': f"Product_{pid}",
                    'preferred_season': random.choice(season_order),
                    'is_guaranteed': False
                }
        
        # 최종 분포 확인
        final_distribution = {}
        guaranteed_distribution = {}
        seasonal_product_count = {"spring": {}, "summer": {}, "autumn": {}, "winter": {}}
        gender_distribution = {}
        
        for product_detail in guaranteed_products.values():
            category = product_detail['category_code']
            preferred_season = product_detail['preferred_season']
            is_guaranteed = product_detail['is_guaranteed']
            gender = product_detail['gender']
            
            main_cat = category_to_main.get(category, 'Unknown')
            sub_cat = category_to_sub.get(category, 'Unknown')
            
            # 전체 분포
            final_distribution[main_cat] = final_distribution.get(main_cat, 0) + 1
            
            # 보장된 제품 분포
            if is_guaranteed:
                guaranteed_distribution[f"{main_cat}>{sub_cat}"] = guaranteed_distribution.get(f"{main_cat}>{sub_cat}", 0) + 1
            
            # 계절별 분포
            seasonal_product_count[preferred_season][main_cat] = seasonal_product_count[preferred_season].get(main_cat, 0) + 1
            
            # 성별 분포
            gender_distribution[gender] = gender_distribution.get(gender, 0) + 1
        
        print("\n" + "-"*50)
        print("제품 데이터의 최종 메인 카테고리 분포")
        print("-"*50)
        total_products = len(guaranteed_products)
        for main_cat in ["상의", "바지", "아우터", "신발"]:
            count = final_distribution.get(main_cat, 0)
            percentage = (count / total_products) * 100
            print(f"  - {main_cat}: {count}개 ({percentage:.1f}%)")
        
        print("\n" + "-"*50)
        print("제품 성별 분포")
        print("-"*50)
        for gender in sorted(gender_distribution.keys()):
            count = gender_distribution[gender]
            percentage = (count / total_products) * 100
            gender_name = {"M": "남성", "F": "여성", "U": "유니섹스"}.get(gender, gender)
            print(f"  - {gender} ({gender_name}): {count}개 ({percentage:.1f}%)")
        
        print(f"\n-- 보장된 카테고리 수: {len(guaranteed_distribution)}개")
        print("보장된 카테고리 (처음 10개):")
        for i, (cat_info, count) in enumerate(list(guaranteed_distribution.items())[:10]):
            print(f"  - {cat_info}: {count}개")
        
        print("\n" + "-"*50)
        print("계절별 제품 분포")
        print("-"*50)
        for season in ["spring", "summer", "autumn", "winter"]:
            season_total = sum(seasonal_product_count[season].values())
            print(f"\n{season.title()} ({season_total}개 제품):")
            for main_cat in ["상의", "바지", "아우터", "신발"]:
                count = seasonal_product_count[season].get(main_cat, 0)
                percentage = (count / season_total * 100) if season_total > 0 else 0
                print(f"    {main_cat}: {count}개 ({percentage:.1f}%)")
        
        return guaranteed_products
    
    def _initialize_seasonal_queries(self) -> Dict[str, List[str]]:
        """계절별 검색어를 category_ranking_dict 기반으로 초기화"""
        seasonal_queries = {
            "spring": [],
            "summer": [],
            "autumn": [],
            "winter": []
        }
        
        # 계절별 카테고리 매핑
        seasonal_mappings = {
            "spring": {
                "상의": ["긴소매 티셔츠", "셔츠/블라우스", "맨투맨/스웨트"],
                "바지": ["데님 팬츠", "코튼 팬츠"],
                "아우터": ["카디건", "나일론/코치 재킷", "환절기 코트"],
                "신발": ["스니커즈", "구두"]
            },
            "summer": {
                "상의": ["반소매 티셔츠", "민소매 티셔츠", "피케/카라 티셔츠"],
                "바지": ["숏 팬츠", "레깅스"],
                "아우터": ["베스트"],
                "신발": ["샌들/슬리퍼", "스니커즈"]
            },
            "autumn": {
                "상의": ["긴소매 티셔츠", "맨투맨/스웨트", "니트/스웨터"],
                "바지": ["데님 팬츠", "코튼 팬츠", "슈트 팬츠/슬랙스"],
                "아우터": ["카디건", "환절기 코트", "나일론/코치 재킷"],
                "신발": ["부츠/워커", "스니커즈"]
            },
            "winter": {
                "상의": ["맨투맨/스웨트", "후드 티셔츠", "니트/스웨터"],
                "바지": ["데님 팬츠", "코튼 팬츠", "트레이닝/조거 팬츠"],
                "아우터": ["롱패딩/헤비 아우터", "숏패딩/헤비 아우터", "겨울 싱글 코드", "무스탕/퍼"],
                "신발": ["부츠/워커", "패딩/퍼 신발"]
            }
        }
        
        for season, season_categories in seasonal_mappings.items():
            for main_category, sub_categories in season_categories.items():
                if main_category in self.category_ranking_dict:
                    for sub_category in sub_categories:
                        if sub_category in self.category_ranking_dict[main_category]:
                            seasonal_queries[season].append(sub_category)
        
        # 각 계절별로 최소 5개 이상의 검색어가 있도록 보장
        for season in seasonal_queries:
            if len(seasonal_queries[season]) < 5:
                all_subcategories = []
                for main_cat, sub_cats in self.category_ranking_dict.items():
                    all_subcategories.extend(list(sub_cats.keys()))
                
                while len(seasonal_queries[season]) < 5 and all_subcategories:
                    random_sub = random.choice(all_subcategories)
                    if random_sub not in seasonal_queries[season]:
                        seasonal_queries[season].append(random_sub)
        
        return seasonal_queries
    
    def _calculate_product_weights(self) -> Dict[str, float]:
        """제품 랭킹 기반 가중치 계산"""
        weights = {}
        
        # product_data에서 필요한 데이터 추출
        product_ids = self.product_data.get('product_id', self.product_data.get('id', []))
        ranks = self.product_data.get('rank', [1] * len(product_ids))
        like_counts = self.product_data.get('like_count', [1] * len(product_ids))
        brand_ids = self.product_data.get('brand_id', self.product_data.get('brand', []))
        
        max_rank = max(ranks) if ranks else 1
        max_like = max(like_counts) if like_counts else 1
        
        if max_rank <= 0: max_rank = 1
        if max_like <= 0: max_like = 1
        
        # 브랜드 가중치 계산
        brand_weights = {}
        if self.brand_data:
            brand_ids_data = self.brand_data.get('brand_id', self.brand_data.get('id', []))
            brand_likes = self.brand_data.get('like_count', [])
            max_brand_like = max(brand_likes) if brand_likes else 1
            
            for bid, like_count in zip(brand_ids_data, brand_likes):
                brand_weights[bid] = like_count / max_brand_like if like_count else 0.1
        
        # 제품별 가중치 계산
        for i, pid in enumerate(product_ids):
            rank = ranks[i] if i < len(ranks) else max_rank
            like_count = like_counts[i] if i < len(like_counts) else 1
            brand_id = brand_ids[i] if i < len(brand_ids) else None
            
            rank_weight = 1 / rank if rank > 0 else 1 / max_rank
            like_weight = like_count / max_like
            brand_weight = brand_weights.get(brand_id, 0.1)
            
            weights[pid] = (rank_weight * 3) + like_weight + brand_weight
        
        return weights
    
    def _initialize_seasonal_weights(self) -> Dict[str, Dict[str, float]]:
        """계절별 카테고리 가중치 초기화"""
        return {
            "spring": {
                "상의": {"반소매 티셔츠": 1.5, "긴소매 티셔츠": 1.3, "셔츠/블라우스": 1.2, "맨투맨/스웨트": 0.8},
                "바지": {"데님 팬츠": 1.2, "코튼 팬츠": 1.3, "숏 팬츠": 0.8},
                "아우터": {"카디건": 1.3, "나일론/코치 재킷": 1.2, "환절기 코트": 1.1},
                "신발": {"스니커즈": 1.2, "구두": 1.1}
            },
            "summer": {
                "상의": {"반소매 티셔츠": 2.0, "민소매 티셔츠": 1.8, "피케/카라 티셔츠": 1.5, "긴소매 티셔츠": 0.3},
                "바지": {"숏 팬츠": 2.0, "데님 팬츠": 0.7, "레깅스": 1.2},
                "아우터": {"나일론/코치 재킷": 0.3, "베스트": 1.5},
                "신발": {"샌들/슬리퍼": 1.8, "스니커즈": 1.3}
            },
            "autumn": {
                "상의": {"긴소매 티셔츠": 1.5, "맨투맨/스웨트": 1.4, "니트/스웨터": 1.3, "반소매 티셔츠": 0.6},
                "바지": {"데님 팬츠": 1.4, "코튼 팬츠": 1.3, "슈트 팬츠/슬랙스": 1.2},
                "아우터": {"카디건": 1.4, "환절기 코트": 1.3, "나일론/코치 재킷": 1.2},
                "신발": {"부츠/워커": 1.3, "스니커즈": 1.2}
            },
            "winter": {
                "상의": {"맨투맨/스웨트": 1.8, "후드 티셔츠": 1.7, "니트/스웨터": 1.6, "반소매 티셔츠": 0.2},
                "바지": {"데님 팬츠": 1.5, "코튼 팬츠": 1.4, "트레이닝/조거 팬츠": 1.3},
                "아우터": {"롱패딩/헤비 아우터": 2.0, "숏패딩/헤비 아우터": 1.8, "겨울 싱글 코드": 1.5, "무스탕/퍼": 1.4},
                "신발": {"부츠/워커": 1.6, "패딩/퍼 신발": 1.4}
            }
        }
    
    def generate_logs_dask(self, n_partitions: int = 16, batch_size: int = 50000, 
                      dashboard_port: int = 8787) -> List[str]:
        """Dask를 활용한 병렬 로그 생성"""
        print("="*60)
        print("Dask 기반 이벤트 로그 생성 시작")
        print("="*60)
        print(f"목표 로그 수: {self.target_log_count:,}개")
        print(f"생성 기간: {self.event_start_date} ~ {self.event_end_date} ({self.total_duration_days}일)")
        
        # 기존 클라이언트 정리
        try:
            current_client = Client.current()
            current_client.close()
            print("기존 Dask 클라이언트 종료")
        except:
            pass
        
        # LocalCluster 생성 (Dashboard 활성화)
        print(f"Dask 클러스터 생성 중... (Dashboard: http://localhost:{dashboard_port}/status)")
        
        cluster = LocalCluster(
            n_workers=min(n_partitions, 8),
            threads_per_worker=1,
            memory_limit='2GB',
            dashboard_address=f':{dashboard_port}',
            silence_logs=False,
            processes=True,
        )
        
        # 클라이언트 생성
        client = Client(cluster)
        
        print("-"*60)
        print("Dask 클러스터 생성 완료!")
        print(f"Dashboard URL: {client.dashboard_link}")
        print(f"Scheduler Address: {client.scheduler_info()['address']}")
        print(f"Workers: {len(client.scheduler_info()['workers'])}")
        print("-"*60)
        
        # 전체 기간을 파티션으로 분할
        days_per_partition = self.total_duration_days // n_partitions
        
        # 파티션별 작업 생성
        futures = []
        for i in range(n_partitions):
            start_day = i * days_per_partition
            end_day = min((i + 1) * days_per_partition, self.total_duration_days) if i < n_partitions - 1 else self.total_duration_days
            
            partition_config = {
                'start_day': start_day,
                'end_day': end_day,
                'partition_id': i,
                'batch_size': batch_size,
                'config': self.config,
                'product_weights': self.product_weights,
                'product_details': self.product_details,
                'seasonal_weights': self.seasonal_weights,
                'seasonal_queries': self.seasonal_queries,
                'category_ranking_dict': self.category_ranking_dict,
                'user_ids': self.user_ids,
                'event_start_date': self.event_start_date,
                'total_duration_days': self.total_duration_days,
                'target_log_count': self.target_log_count,
                'promotion_data': self.promotion_data
            }
            
            future = client.submit(generate_partition_logs_with_guaranteed_coverage, partition_config)
            futures.append(future)
            print(f"파티션 {i} 작업 제출 완료 (일수: {start_day}~{end_day})")
        
        print("\n" + "="*60)
        print("모든 작업 제출 완료. Dashboard에서 진행 상황을 확인하세요!")
        print(f"Dashboard: {client.dashboard_link}")
        print("="*60)
        
        # 결과 수집
        results = []
        try:
            print("작업 결과 수집 중...")
            results = client.gather(futures)
            print("-- 모든 작업 완료!")
        except Exception as e:
            print(f"ERROR: 작업 실행 중 오류: {e}")
            results = []
        finally:
            # 개선된 클러스터 정리 과정
            print("Dask 클러스터 정리 중...")
            try:
                # 1. 모든 작업이 완료될 때까지 대기
                client.wait_for_workers(0, timeout=30)
                
                # 2. 클라이언트 정리
                client.close()
                
                # 3. 잠시 대기 (연결 해제 시간 확보)
                import time
                time.sleep(2)
                
                # 4. 클러스터 정리
                cluster.close()
                
            except Exception as cleanup_error:
                print(f"클러스터 정리 중 무시할 수 있는 에러: {cleanup_error}")
                # 강제 종료
                try:
                    cluster.close()
                except:
                    pass
        
        # 결과 파일 경로 반환
        partition_files = []
        for result in results:
            if result:
                partition_files.extend(result)
        
        print(f"총 {len(partition_files)}개 파티션 파일 생성 완료")
        return partition_files

In [9]:
def generate_page_info(event_type: str, user_details: Dict, **properties) -> Dict:
    """페이지 정보 생성 함수"""
    base_url = "https://high-five.store"
    
    # 이벤트 타입별 페이지 URL 매핑
    page_mapping = {
        "view_main": {
            "url": f"{base_url}/",
            "title": "메인페이지"
        },
        "search": {
            "url": f"{base_url}/search?q={properties.get('query', 'default')}",
            "title": "검색결과"
        },
        "product_view": {
            "url": f"{base_url}/products/{properties.get('product_id', '1')}",
            "title": f"상품상세 - {properties.get('product_name', '상품')}"
        },
        "promotion_view": {
            "url": f"{base_url}/promotions/{properties.get('promotion_id', '1')}",
            "title": f"프로모션 - {properties.get('promotion_id', '1')}"
        },
        "promotion_click": {
            "url": f"{base_url}/promotions/{properties.get('promotion_id', '1')}",
            "title": f"프로모션 - {properties.get('promotion_id', '1')}"
        },
        "add_to_cart": {
            "url": f"{base_url}/cart",
            "title": "장바구니"
        },
        "purchase": {
            "url": f"{base_url}/checkout/complete",
            "title": "구매완료"
        },
        "order": {
            "url": f"{base_url}/orders/{properties.get('order_id', 'new')}",
            "title": "주문완료"
        },
        "product_like": {
            "url": f"{base_url}/products/{properties.get('product_id', '1')}",
            "title": f"상품상세 - {properties.get('product_name', '상품')}"
        },
        "brand_like": {
            "url": f"{base_url}/brands/{properties.get('brand_id', '1')}",
            "title": f"브랜드 - {properties.get('brand_id', '1')}"
        },
        "change_cart_item_quantity": {
            "url": f"{base_url}/cart",
            "title": "장바구니"
        },
        "remove_from_cart": {
            "url": f"{base_url}/cart",
            "title": "장바구니"
        },
        "refund": {
            "url": f"{base_url}/orders/refund",
            "title": "환불신청"
        }
    }
    
    page_info = page_mapping.get(event_type, {
        "url": f"{base_url}/",
        "title": "메인페이지"
    })
    
    return page_info

def generate_landing_url(user_details: Dict, first_event_type: str = None) -> str:
    """세션 최초 방문 페이지 URL 생성 - 첫 번째 이벤트 타입 기반"""
    base_url = "https://high-five.store"
    
    # 첫 번째 이벤트 타입에 따른 랜딩 URL 결정
    if first_event_type == "view_main":
        return base_url + "/"
    elif first_event_type == "product_view":
        return "https://google.com"
    else:
        # 기타 경우 (현재는 거의 모든 세션이 view_main으로 시작하므로 기본값)
        return base_url + "/"

def get_referrer_url(current_url: str, previous_url: str = None) -> str:
    """내부 referrer URL 생성"""
    if previous_url:
        return previous_url
    
    base_url = "https://high-five.store"
    
    # 현재 페이지 타입에 따른 일반적인 referrer 패턴
    if "/products/" in current_url:
        # 상품 페이지는 보통 검색이나 카테고리에서 옴
        referrer_options = [
            f"{base_url}/search",
            f"{base_url}/categories/tops",
            f"{base_url}/categories/bottoms",
            f"{base_url}/",
            None  # 직접 접근
        ]
        weights = [0.3, 0.2, 0.2, 0.2, 0.1]
    elif "/search" in current_url:
        # 검색 페이지는 보통 메인에서 옴
        referrer_options = [f"{base_url}/", None]
        weights = [0.8, 0.2]
    elif "/cart" in current_url:
        # 장바구니는 상품 페이지에서 옴
        referrer_options = [f"{base_url}/products/sample", f"{base_url}/"]
        weights = [0.7, 0.3]
    else:
        # 기타 페이지
        referrer_options = [f"{base_url}/", None]
        weights = [0.6, 0.4]
    
    return random.choices(referrer_options, weights=weights)[0]

def create_event(event_type: str, user_details: Dict, session_id: str, 
                event_time: datetime, previous_url: str = None, 
                is_first_event: bool = False, **properties) -> Dict[str, Any]:
    """수정된 이벤트 생성 함수 - 컬럼 순서 조정"""
    import uuid
    from datetime import timedelta
    
    ingest_time = event_time + timedelta(seconds=random.randint(1, 10))
    
    # 페이지 정보 생성
    page_info = generate_page_info(event_type, user_details, **properties)
    current_url = page_info["url"]
    page_title = page_info["title"]
    
    # 세션별 landing_url 관리 (세션당 한 번만 생성)
    if not hasattr(create_event, 'session_landing_urls'):
        create_event.session_landing_urls = {}
    
    if session_id not in create_event.session_landing_urls:
        # 첫 번째 이벤트인 경우 해당 이벤트 타입을 기반으로 랜딩 URL 생성
        if is_first_event:
            create_event.session_landing_urls[session_id] = generate_landing_url(user_details, event_type)
        else:
            create_event.session_landing_urls[session_id] = generate_landing_url(user_details)
    
    landing_url = create_event.session_landing_urls[session_id]
    
    # Referrer URL 생성
    referrer_url = get_referrer_url(current_url, previous_url)
    
    # 원하는 컬럼 순서대로 이벤트 구조 생성
    event = {
        "event_id": str(uuid.uuid4()),
        "anonymous_id": user_details["anonymous_id"],
        "user_id": user_details["user_id"],
        "session_id": session_id,
        "event_type": event_type,
        "utm_params": None,  # 일단 Null 값으로 설정
        "page_url": current_url,
        "page_view": page_title if event_type != "view_main" else None,  # 메인페이지는 None
        "page_referrer": referrer_url,
        "landing_url": landing_url,
        "event_timestamp": event_time.isoformat(),
        "ingest_timestamp": ingest_time.isoformat()
    }
    
    # 이벤트 속성 추가 (마지막에 추가)
    if properties and event_type != "view_main":
        event["event_properties"] = properties
    else:
        event["event_properties"] = None
    
    return event

# 세션별 페이지 이동 추적을 위한 클래스
class SessionPageTracker:
    def __init__(self):
        self.session_pages = {}  # session_id -> [page_urls]
    
    def add_page(self, session_id: str, page_url: str):
        if session_id not in self.session_pages:
            self.session_pages[session_id] = []
        self.session_pages[session_id].append(page_url)
    
    def get_previous_page(self, session_id: str) -> str:
        if session_id in self.session_pages and len(self.session_pages[session_id]) > 0:
            return self.session_pages[session_id][-1]
        return None

# 전역 페이지 트래커 인스턴스
page_tracker = SessionPageTracker()

In [10]:
def generate_partition_logs_with_guaranteed_coverage(partition_config: Dict) -> List[str]:
    """모든 카테고리가 최소 1번은 나오도록 보장하는 파티션 로그 생성 함수 - 완전 수정 버전"""
    import gc
    import json
    import random
    import uuid
    import pandas as pd
    from datetime import datetime, timedelta, timezone, date
    
    start_day = partition_config['start_day']
    end_day = partition_config['end_day']
    partition_id = partition_config['partition_id']
    batch_size = partition_config['batch_size']
    config = partition_config['config']
    product_weights = partition_config['product_weights']
    product_details = partition_config['product_details']
    seasonal_weights = partition_config['seasonal_weights']
    seasonal_queries = partition_config['seasonal_queries']
    category_ranking_dict = partition_config['category_ranking_dict']
    user_ids = partition_config['user_ids']
    event_start_date = partition_config['event_start_date']
    total_duration_days = partition_config['total_duration_days']
    target_log_count = partition_config['target_log_count']
    promotion_data = partition_config.get('promotion_data', {})
    
    print(f"파티션 {partition_id}: {start_day}일~{end_day}일 처리 시작")
    print(f"파티션 {partition_id}: 프로모션 데이터 {len(promotion_data.get('brand_id', []))}개")
    
    partition_events = []
    local_registered_users = set()
    
    # 파티션별 카테고리 사용 추적
    used_categories = set()
    guaranteed_products = {pid: details for pid, details in product_details.items() 
                          if details.get('is_guaranteed', False)}
    
    print(f"파티션 {partition_id}: 보장된 제품 {len(guaranteed_products)}개")


    # 세션별 페이지 이동 추적을 위한 클래스
    class SessionPageTracker:
        def __init__(self):
            self.session_pages = {}  # session_id -> [page_urls]
        
        def add_page(self, session_id: str, page_url: str):
            if session_id not in self.session_pages:
                self.session_pages[session_id] = []
            self.session_pages[session_id].append(page_url)
        
        def get_previous_page(self, session_id: str) -> str:
            if session_id in self.session_pages and len(self.session_pages[session_id]) > 0:
                return self.session_pages[session_id][-1]
            return None

    # 전역 페이지 트래커 인스턴스
    page_tracker = SessionPageTracker()
        
    def generate_page_info(event_type: str, user_details: Dict, **properties) -> Dict:
        """페이지 정보 생성 함수 - 데이터프레임 참조"""
        base_url = "https://high-five.store"
        
        # 이벤트 타입별 페이지 URL 및 제목 매핑
        if event_type == "view_main":
            return {
                "url": f"{base_url}/",
                "title": "하이파이브"
            }
        elif event_type == "search":
            query = properties.get('query', '')
            title = f"{query} | 하이파이브" if query else "검색 | 하이파이브"
            return {
                "url": f"{base_url}/search?q={query if query else 'default'}",
                "title": title
            }
        elif event_type == "product_view":
            product_id = properties.get('product_id', '')
            product_name = ""
            
            # product_iso_df에서 상품명 조회
            try:
                if product_id and 'product_iso_df' in globals():
                    product_row = product_iso_df[product_iso_df['id'] == product_id]
                    if not product_row.empty:
                        product_name = product_row.iloc[0].get('product_name', '')
            except Exception as e:
                print(f"상품 정보 조회 실패 (product_id: {product_id}): {e}")
            
            title = f"{product_name} | 하이파이브" if product_name else "하이파이브"
            return {
                "url": f"{base_url}/products/{product_id}",
                "title": title
            }
        elif event_type == "promotion_view":
            return {
                "url": f"{base_url}/",  # 메인페이지에서 프로모션 배너 조회
                "title": "하이파이브"
            }
        elif event_type == "promotion_click":
            promotion_id = properties.get('promotion_id', '')
            promotion_title = ""
            
            # promotion_iso_df에서 프로모션 제목 조회
            try:
                if promotion_id and 'promotion_iso_df' in globals():
                    promo_row = promotion_iso_df[promotion_iso_df['brand_id'] == promotion_id]
                    if not promo_row.empty:
                        promotion_title = promo_row.iloc[0].get('title', '')
            except Exception as e:
                print(f"프로모션 정보 조회 실패 (promotion_id: {promotion_id}): {e}")
            
            if promotion_title:
                title = f"{promotion_title} 상품 | 하이파이브"
            else:
                title = f"Brand {promotion_id} 상품 목록 | 하이파이브"
            
            return {
                "url": f"{base_url}/",  # 메인페이지에서 프로모션 배너 클릭
                "title": title
            }
        elif event_type in ["add_to_cart", "change_cart_item_quantity", "remove_from_cart"]:
            return {
                "url": f"{base_url}/cart",
                "title": "장바구니 | 하이파이브"
            }
        elif event_type == "order":
            return {
                "url": f"{base_url}/orders/{properties.get('order_id', 'new')}",
                "title": "하이파이브"
            }
        elif event_type == "product_like":
            return {
                "url": f"{base_url}/products/{properties.get('product_id', '1')}",
                "title": "하이파이브"
            }
        elif event_type == "brand_like":
            return {
                "url": f"{base_url}/brands/{properties.get('brand_id', '1')}",
                "title": "하이파이브"
            }
        elif event_type == "refund":
            return {
                "url": f"{base_url}/orders/refund",
                "title": "하이파이브"
            }
        else:
            combined_df = pd.DataFrame(all_events)
        
        # 최종 파일 저장
        output_file = os.path.join(OUTPUT_LOG_PATH, 'event_logs_10m.parquet')
        
        if os.path.exists(output_file):
            print(f"   기존 파일 삭제: {output_file}")
            os.remove(output_file)
        
        print(f"   최종 파일 저장 중: {output_file}")
        combined_df.to_parquet(output_file, index=False)
        
        # 임시 파일 정리
        for temp_file in temp_files:
            temp_path = os.path.join(OUTPUT_LOG_PATH, temp_file)
            if os.path.exists(temp_path):
                os.remove(temp_path)
        
        # 최종 통계
        file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
        
        print("=" * 70)
        print("🎉 올바른 스키마 이벤트 로그 생성 완료!")
        print("=" * 70)
        print(f"📁 저장 위치: {output_file}")
        print(f"📊 총 이벤트 수: {len(combined_df):,}개")
        print(f"💽 파일 크기: {file_size_mb:.2f} MB")
        print(f"📅 시뮬레이션 기간: {total_days}일")
        print(f"👥 고유 사용자: {combined_df['user_id'].nunique():,}명")
        print(f"🎯 이벤트 타입: {combined_df['event_name'].nunique()}개")
        print(f"🕐 현실적인 시간대 분포 적용됨!")
        
        # 스키마 검증
        print(f"\n🔍 올바른 스키마 검증:")
        required_columns = ['event_name', 'event_id', 'user_id', 'anonymous_id', 'session_id', 'context', 'event_properties', 'timestamp']
        
        missing_columns = [col for col in required_columns if col not in combined_df.columns]
        if missing_columns:
            print(f"❌ 누락된 컬럼: {missing_columns}")
        else:
            print("✅ 모든 필수 컬럼 존재")
        
        print(f"\n📋 생성된 컬럼:")
        for col in combined_df.columns:
            print(f"   - {col}: {combined_df[col].dtype}")
        
        return combined_df
        
    except Exception as e:
        print(f"   ❌ 최종 저장 오류: {e}")
        return None

    # 헬퍼 함수들
    def get_season_from_date(current_date: date) -> str:
        month = current_date.month
        if month in [3, 4, 5]: return "spring"
        elif month in [6, 7, 8]: return "summer"
        elif month in [9, 10, 11]: return "autumn"
        else: return "winter"
    
    def get_phase_from_day(day_offset: int) -> str:
        early_end = total_duration_days // 3
        growth_end = total_duration_days * 2 // 3
        if day_offset <= early_end: return "EARLY"
        elif day_offset <= growth_end: return "GROWTH"
        else: return "MATURE"
    
    def calculate_daily_dau(day_offset: int) -> int:
        avg_events_per_session = 8
        total_sessions_needed = target_log_count // avg_events_per_session
        base_start_dau = max(total_sessions_needed // total_duration_days // 2, 1)
        base_end_dau = max(total_sessions_needed // total_duration_days * 2, 1)
        daily_dau = base_start_dau + (base_end_dau - base_start_dau) * (day_offset / total_duration_days)
        
        current_date = event_start_date + timedelta(days=day_offset)
        if current_date.weekday() >= 5:
            daily_dau *= 1.2
        
        return max(int(daily_dau), 5)
    
    def get_seasonal_category_weights(current_date: date) -> tuple:
        season = get_season_from_date(current_date)
        season_weights = seasonal_weights[season]
        category_weights = {}
        
        for main_category, sub_categories in category_ranking_dict.items():
            main_weight = season_weights.get(main_category, {})
            for sub_category, code in sub_categories.items():
                weight = main_weight.get(sub_category, 1.0)
                category_weights[code] = weight
        
        return category_weights, season
    
    def assign_user_local(current_phase: str) -> Dict[str, Any]:
        """수정된 사용자 할당 - 처음부터 로그인 30%, 로그인 전환 로직 개선"""
        phase_config = config["PHASE_SETTINGS"][current_phase]
        conversion_rate = phase_config["anonymous_to_member_conversion_rate"]
        returning_user_rates = {"EARLY": 0.3, "GROWTH": 0.6, "MATURE": 0.7}
        returning_rate = returning_user_rates.get(current_phase, 0.5)
        
        # 처음부터 로그인된 비율을 30%로 설정
        initial_login_rate = 0.30
        
        user_details = {
            "user_id": None,
            "anonymous_id": str(uuid.uuid4()),
            "is_new_user": True,
            "converted_in_session": False,
            "will_convert_to": None
        }
        
        # 1. 처음부터 로그인된 사용자 (30% 확률)
        if random.random() < initial_login_rate:
            if local_registered_users and random.random() < returning_rate:
                user_details["user_id"] = random.choice(list(local_registered_users))
                user_details["is_new_user"] = False
            else:
                available_users = set(user_ids) - local_registered_users
                if available_users:
                    new_user_id = random.choice(list(available_users))
                    user_details["user_id"] = new_user_id
                    local_registered_users.add(new_user_id)
        
        # 2. 비로그인으로 시작하지만 세션 중 로그인할 가능성 (70% 확률)
        else:
            # 세션 중 로그인 전환 여부 결정
            if random.random() < conversion_rate:
                available_users = set(user_ids) - local_registered_users
                if available_users:
                    new_user_id = random.choice(list(available_users))
                    user_details["will_convert_to"] = new_user_id
                    user_details["converted_in_session"] = True  # 전환 예정 표시
        
        return user_details
    
    def assign_segment(segment_distribution: Dict[str, float]) -> str:
        segments = list(segment_distribution.keys())
        weights = list(segment_distribution.values())
        return random.choices(segments, weights=weights, k=1)[0]
    
    def weighted_product_choice_with_guaranteed_coverage(category_weights: Dict[str, float] = None, 
                                                       current_season: str = None,
                                                       force_unused: bool = False) -> str:
        if force_unused or random.random() < 0.3:
            unused_guaranteed = []
            for pid, details in guaranteed_products.items():
                if details['category_code'] not in used_categories:
                    unused_guaranteed.append(pid)
            
            if unused_guaranteed:
                selected = random.choice(unused_guaranteed)
                used_categories.add(product_details[selected]['category_code'])
                return selected
        
        if not category_weights:
            product_ids = list(product_weights.keys())
            weights = list(product_weights.values())
            return random.choices(product_ids, weights=weights, k=1)[0]
        
        adjusted_weights = {}
        for product_id, base_weight in product_weights.items():
            product_detail = product_details.get(product_id, {})
            category_code = product_detail.get('category_code', '')
            preferred_season = product_detail.get('preferred_season', current_season)
            
            seasonal_weight = category_weights.get(category_code, 1.0)
            season_match_bonus = 1.5 if preferred_season == current_season else 1.0
            
            adjusted_weights[product_id] = base_weight * seasonal_weight * season_match_bonus
        
        if not adjusted_weights:
            return random.choice(list(product_weights.keys()))
        
        product_ids = list(adjusted_weights.keys())
        weights = list(adjusted_weights.values())
        selected = random.choices(product_ids, weights=weights, k=1)[0]
        
        if selected in product_details:
            used_categories.add(product_details[selected]['category_code'])
        
        return selected
    
    def generate_seasonal_search_query_with_optional_filters(current_date: date, category_ranking_dict: Dict) -> tuple:
        season = get_season_from_date(current_date)
        
        if random.random() < 0.7 and season in seasonal_queries:
            if seasonal_queries[season]:
                query = random.choice(seasonal_queries[season])
            else:
                query = random.choice(["상의", "바지", "아우터", "신발"])
        else:
            all_subcategories = []
            for main_category, sub_categories in category_ranking_dict.items():
                all_subcategories.extend(list(sub_categories.keys()))
            query = random.choice(all_subcategories) if all_subcategories else "상의"
        
        filters = {}
        
        if random.random() < 0.7:
            filters["category"] = random.choice(list(category_ranking_dict.keys()))
        
        if random.random() < 0.4:
            gender_weights = {"M": 0.6, "F": 0.15, "U": 0.25}
            genders = list(gender_weights.keys())
            weights = list(gender_weights.values())
            filters["gender"] = random.choices(genders, weights=weights, k=1)[0]
        
        return query, filters
    
    def get_active_promotions(current_date):
        """현재 날짜에 활성화된 프로모션만 반환 - 기본값 강제 추가 안함"""
        from datetime import datetime
        import pandas as pd
        
        if not promotion_data or not promotion_data.get('brand_id'):
            # 프로모션 데이터가 없으면 빈 리스트 반환
            return []
        
        active_promotions = []
        
        try:
            brand_ids = promotion_data.get('brand_id', [])
            start_dates = promotion_data.get('start_date', [])
            end_dates = promotion_data.get('end_date', [])
            
            for i in range(len(brand_ids)):
                try:
                    brand_id = brand_ids[i]
                    
                    # 시작일/종료일이 있는 경우에만 날짜 체크
                    if i < len(start_dates) and i < len(end_dates):
                        start_date_str = start_dates[i]
                        end_date_str = end_dates[i]
                        
                        if 'T' in str(start_date_str):
                            start_date = pd.to_datetime(start_date_str).date()
                        else:
                            start_date = datetime.strptime(str(start_date_str), '%Y-%m-%d').date()
                        
                        if 'T' in str(end_date_str):
                            end_date = pd.to_datetime(end_date_str).date()
                        else:
                            end_date = datetime.strptime(str(end_date_str), '%Y-%m-%d').date()
                        
                        # 현재 날짜가 프로모션 기간 내인 경우만 추가
                        if start_date <= current_date <= end_date:
                            active_promotions.append({
                                'brand_id': brand_id,
                                'start_date': start_date_str,
                                'end_date': end_date_str
                            })
                    else:
                        # 날짜 정보가 없는 프로모션은 제외
                        continue
                        
                except Exception as e:
                    print(f"프로모션 {brand_id} 처리 중 오류: {e}")
                    continue
            
            # 실제로 활성화된 프로모션만 반환 (기본값 추가 안함)
            return active_promotions
            
        except Exception as e:
            print(f"get_active_promotions 전체 오류: {e}")
            return []
        
    def get_products_by_brand(brand_id: str) -> List[Dict]:
        brand_products = []
        for pid, details in product_details.items():
            if details.get('brand_id') == brand_id:
                brand_products.append(details)
        return brand_products
    
    def get_user_cart_items(user_id: str) -> List[Dict]:
        if not hasattr(get_user_cart_items, 'user_carts'):
            get_user_cart_items.user_carts = {}
        return get_user_cart_items.user_carts.get(user_id, [])
    
    def save_user_order(user_id: str, order_info: Dict):
        if not hasattr(save_user_order, 'user_orders'):
            save_user_order.user_orders = {}
        
        if user_id not in save_user_order.user_orders:
            save_user_order.user_orders[user_id] = []
        
        save_user_order.user_orders[user_id].append(order_info)
    
    def get_user_orders(user_id: str) -> List[Dict]:
        if not hasattr(save_user_order, 'user_orders'):
            return []
        return save_user_order.user_orders.get(user_id, [])
    
    def get_product_detail_from_df(product_id: str) -> Dict:
        product_detail = product_details.get(product_id)
        if product_detail:
            return product_detail
        else:
            return {
                'id': product_id,
                'product_name': f'Product_{product_id}',
                'category_code': 'Unknown',
                'price': 50000,
                'discounted_price': 40000,
                'gender': 'U',
                'brand_id': 'BRAND_DEFAULT'
            }
        
    def generate_events_for_session(segment: str, user_details: Dict, 
                                session_id: str, current_date: date,
                                category_weights: Dict[str, float],
                                current_season: str,
                                session_count: int) -> List[Dict]:
        events = []
        segment_behavior = config["SEGMENT_BEHAVIORS"][segment]
        action_probs = segment_behavior["action_probabilities"]
        action_counts = segment_behavior["action_counts"]
        
        # UTC 기준 시간 생성
        base_time = datetime.combine(current_date, datetime.min.time()).replace(tzinfo=timezone.utc)
        session_start = base_time + timedelta(seconds=random.randint(0, 86400))
        current_time = session_start
        
        force_unused = session_count < 100
        
        # 로그인 전환 관련 변수
        login_conversion_point = None
        will_convert_user_id = None
        converted_already = False
        
        if user_details.get("will_convert_to"):
            login_conversion_point = random.randint(3, 8)
            will_convert_user_id = user_details["will_convert_to"]
        
        event_count = 0
        
        # 수정된 로그인 전환 체크 함수 - login 이벤트 생성하지 않음
        def check_and_execute_login_conversion():
            nonlocal event_count, converted_already
            if (login_conversion_point and event_count >= login_conversion_point and 
                will_convert_user_id and not user_details.get("user_id") and not converted_already):
                
                # login 이벤트 생성하지 않고, user_details만 업데이트
                user_details["user_id"] = will_convert_user_id
                user_details.pop("will_convert_to", None)
                local_registered_users.add(will_convert_user_id)
                converted_already = True
                
                # print(f"세션 {session_id[:8]}: 익명 사용자가 회원으로 전환됨 (user_id: {will_convert_user_id})")
                return True
            return False
        
        # 1. 랜딩 이벤트 (무조건 생성)
        main_page_event = create_event(
            "view_main", user_details, session_id, current_time,
            is_first_event=True  # 첫 번째 이벤트임을 표시
        )
        events.append(main_page_event)
        page_tracker.add_page(session_id, main_page_event["page_url"])
        current_time += timedelta(seconds=random.randint(2, 5))
        event_count += 1

        # 1.1. promotion_view 이벤트 - 실제 활성화된 프로모션만 사용
        try:
            active_promotions = get_active_promotions(current_date)
            
            # 실제 활성화된 프로모션만 사용 (강제로 3개 맞추지 않음)
            if active_promotions and len(active_promotions) > 0:
                # 실제 활성화된 프로모션들로만 section_id 생성
                session_section_id = "_".join([str(promo['brand_id']) for promo in active_promotions])
                
                # 실제 활성화된 프로모션 개수만큼만 promotion_view 생성
                for i, promotion in enumerate(active_promotions):
                    promotion_view_event = create_event(
                        "promotion_view", user_details, session_id, current_time,
                        previous_url=page_tracker.get_previous_page(session_id),
                        promotion_id=promotion['brand_id'],
                        location="main_triple_banner",
                        section_id=session_section_id,
                        position=i + 1
                    )
                    events.append(promotion_view_event)
                    page_tracker.add_page(session_id, promotion_view_event["page_url"])
                    current_time += timedelta(milliseconds=random.randint(100, 500))
                    event_count += 1
                
                # 1.2. promotion_click (실제 활성화된 프로모션 중에서만 클릭)
                if random.random() < action_probs.get("promotion_click", 0.2):
                    clicked_promotion = random.choice(active_promotions)
                    clicked_position = active_promotions.index(clicked_promotion) + 1
                    
                    promotion_click_event = create_event(
                        "promotion_click", user_details, session_id, current_time,
                        previous_url=page_tracker.get_previous_page(session_id),
                        promotion_id=clicked_promotion['brand_id'],
                        location="main_triple_banner",
                        section_id=session_section_id,
                        position=clicked_position
                    )
                    events.append(promotion_click_event)
                    page_tracker.add_page(session_id, promotion_click_event["page_url"])
                    current_time += timedelta(seconds=random.randint(1, 3))
                    event_count += 1
                    
                    # 1.3. 클릭한 프로모션의 브랜드 제품 조회
                    brand_products = get_products_by_brand(clicked_promotion['brand_id'])
                    if brand_products:
                        promo_product = random.choice(brand_products)
                        
                        # 프로모션 클릭 후 상품 조회 시 특별한 referrer 설정
                        brand_referrer = f"https://high-five.store/brands/{clicked_promotion['brand_id']}"
                        
                        promo_product_view = create_event(
                            "product_view", user_details, session_id, current_time,
                            previous_url=brand_referrer,  # 브랜드 페이지를 referrer로 설정
                            product_id=promo_product['id'],
                            product_name=promo_product['product_name'],
                            category_code=promo_product['category_code'],
                            price=promo_product['price'],
                            discounted_price=promo_product['discounted_price'],
                            gender=promo_product['gender']
                        )
                        events.append(promo_product_view)
                        page_tracker.add_page(session_id, promo_product_view["page_url"])
                        current_time += timedelta(seconds=random.randint(10, 30))
                        event_count += 1
                        
            else:
                # 활성화된 프로모션이 없으면 promotion_view 이벤트 생성하지 않음
                print(f"날짜 {current_date}에 활성화된 프로모션이 없음 - promotion_view 이벤트 생성 안함")
                
        except Exception as e:
            # promotion_view 생성 실패 시에도 로그만 남기고 억지로 생성하지 않음
            print(f"세션 {session_id[:8]}에서 promotion_view 생성 실패: {e}")
            print(f"날짜 {current_date}에 활성화된 프로모션 조회 실패 - promotion_view 이벤트 생성 안함")
        
        # 2. 검색 행동
        if random.random() < action_probs["search"]:
            max_searches = action_counts.get("max_searches", 3)
            num_searches = random.randint(1, max_searches)
            
            for _ in range(num_searches):
                check_and_execute_login_conversion()
                
                search_query, search_filters = generate_seasonal_search_query_with_optional_filters(
                    current_date, category_ranking_dict
                )
                
                event_properties = {
                    "query": search_query,
                    "results_count": random.randint(5, 50)
                }
                
                if search_filters:
                    event_properties["filters"] = search_filters
                
                search_event = create_event(
                    "search", user_details, session_id, current_time,
                    previous_url=page_tracker.get_previous_page(session_id),
                    **event_properties
                )
                events.append(search_event)
                page_tracker.add_page(session_id, search_event["page_url"])
                current_time += timedelta(seconds=random.randint(10, 60))
                event_count += 1
        
        # 3. 상품 조회 행동
        viewed_products = []
        if random.random() < action_probs["product_view"]:
            max_views = action_counts.get("max_product_views", 10)
            num_views = random.randint(1, max_views)
            
            for view_idx in range(num_views):
                check_and_execute_login_conversion()
                
                force_unused_for_this_view = force_unused and view_idx == 0
                
                product_id = weighted_product_choice_with_guaranteed_coverage(
                    category_weights, current_season, force_unused_for_this_view
                )
                product_detail = get_product_detail_from_df(product_id)
                
                view_event = create_event(
                    "product_view", user_details, session_id, current_time,
                    previous_url=page_tracker.get_previous_page(session_id),
                    product_id=product_detail["id"],
                    product_name=product_detail["product_name"],
                    category_code=product_detail["category_code"],
                    price=product_detail["price"],
                    discounted_price=product_detail["discounted_price"],
                    gender=product_detail["gender"]
                )
                events.append(view_event)
                page_tracker.add_page(session_id, view_event["page_url"])
                viewed_products.append(product_detail)
                current_time += timedelta(seconds=random.randint(15, 120))
                event_count += 1
        
        # 4. 좋아요 행동
        if user_details["user_id"] and random.random() < action_probs.get("like", 0):
            if viewed_products:
                max_likes = action_counts.get("max_likes", 5)
                num_likes = min(random.randint(1, max_likes), len(viewed_products))
                
                for liked_product in random.sample(viewed_products, num_likes):
                    is_liked = random.choice([True, False])
                    
                    like_event = create_event(
                        "product_like", user_details, session_id, current_time,
                        previous_url=page_tracker.get_previous_page(session_id),
                        product_id=liked_product["id"],
                        is_liked=is_liked
                    )
                    events.append(like_event)
                    page_tracker.add_page(session_id, like_event["page_url"])
                    current_time += timedelta(seconds=random.randint(5, 30))
                    event_count += 1
        
        # 4.1. 브랜드 좋아요 행동
        if user_details["user_id"] and random.random() < action_probs.get("brand_like", 0.1):
            if viewed_products:
                brand_ids = list(set([p.get("brand_id") for p in viewed_products if p.get("brand_id")]))
                if brand_ids:
                    selected_brand = random.choice(brand_ids)
                    is_liked = random.choice([True, False])
                    
                    brand_like_event = create_event(
                        "brand_like", user_details, session_id, current_time,
                        previous_url=page_tracker.get_previous_page(session_id),
                        brand_id=selected_brand,
                        is_liked=is_liked
                    )
                    events.append(brand_like_event)
                    page_tracker.add_page(session_id, brand_like_event["page_url"])
                    current_time += timedelta(seconds=random.randint(5, 30))
                    event_count += 1
        
        # 5. 장바구니 행동
        cart_items = []
        user_cart_items = get_user_cart_items(user_details["user_id"]) if user_details["user_id"] else []
        
        if random.random() < action_probs.get("add_to_cart", 0):
            if user_details["user_id"] and viewed_products:
                max_cart = action_counts.get("max_cart_additions", 3)
                num_cart = min(random.randint(1, max_cart), len(viewed_products))
                
                for cart_product in random.sample(viewed_products, num_cart):
                    check_and_execute_login_conversion()
                    
                    quantity = random.randint(1, 3)
                    cart_event = create_event(
                        "add_to_cart", user_details, session_id, current_time,
                        previous_url=page_tracker.get_previous_page(session_id),
                        product_id=cart_product["id"],
                        quantity=quantity,
                        discounted_price=cart_product["discounted_price"]
                    )
                    events.append(cart_event)
                    page_tracker.add_page(session_id, cart_event["page_url"])
                    
                    cart_item = {
                        "product_id": cart_product["id"],
                        "quantity": quantity,
                        "discounted_price": cart_product["discounted_price"],
                        "added_at": current_time
                    }
                    cart_items.append(cart_item)
                    user_cart_items.append(cart_item)
                    
                    current_time += timedelta(seconds=random.randint(30, 180))
                    event_count += 1
                    
                    # 5.1. 장바구니 수량 변경
                    if random.random() < 0.2:
                        new_quantity = random.randint(1, 5)
                        change_quantity_event = create_event(
                            "change_cart_item_quantity", user_details, session_id, current_time,
                            previous_url=page_tracker.get_previous_page(session_id),
                            product_id=cart_product["id"],
                            old_quantity=quantity,
                            new_quantity=new_quantity,
                            discounted_price=cart_product["discounted_price"]
                        )
                        events.append(change_quantity_event)
                        page_tracker.add_page(session_id, change_quantity_event["page_url"])
                        cart_item["quantity"] = new_quantity
                        current_time += timedelta(seconds=random.randint(10, 60))
                        event_count += 1
                    
                    # 5.2. 장바구니에서 제거
                    if random.random() < 0.1:
                        remove_cart_event = create_event(
                            "remove_from_cart", user_details, session_id, current_time,
                            previous_url=page_tracker.get_previous_page(session_id),
                            product_id=cart_product["id"],
                            quantity=cart_item["quantity"],
                            discounted_price=cart_product["discounted_price"]
                        )
                        events.append(remove_cart_event)
                        page_tracker.add_page(session_id, remove_cart_event["page_url"])
                        if cart_item in cart_items:
                            cart_items.remove(cart_item)
                        if cart_item in user_cart_items:
                            user_cart_items.remove(cart_item)
                        current_time += timedelta(seconds=random.randint(5, 30))
                        event_count += 1
        
        # 6. 주문 행동
        if user_details["user_id"] and (cart_items or user_cart_items) and random.random() < action_probs.get("order", 0):
            all_cart_items = cart_items + [item for item in user_cart_items if item not in cart_items]
            
            if all_cart_items:
                order_items = random.sample(all_cart_items, random.randint(1, min(len(all_cart_items), 3)))
                total_price = sum(item["discounted_price"] * item["quantity"] for item in order_items)
                
                order_status = random.choices(
                    ["paid", "shipping", "shipped", "cancelled", "pending_paid"],
                    weights=[0.7, 0.1, 0.1, 0.05, 0.05],
                    k=1
                )[0]
                
                if order_status == "paid":
                    payment_status = "paid_kakao"
                elif order_status == "cancelled":
                    payment_status = "cancelled_kakao"
                elif order_status == "pending_paid":
                    payment_status = random.choice([
                        "pending_payment", "pending_kakao_ready", "ready_kakao", 
                        "failed_kakao_ready", "failed_kakao_ready_response", "failed_kakao_approve"
                    ])
                else:
                    payment_status = "paid_kakao"
                
                order_event = create_event(
                    "order", user_details, session_id, current_time,
                    previous_url=page_tracker.get_previous_page(session_id),
                    user_id=user_details["user_id"],
                    status=order_status,
                    order_items=[{
                        "product_id": item["product_id"],
                        "quantity": item["quantity"],
                        "discounted_price": item["discounted_price"]
                    } for item in order_items],
                    is_from_cart=True,
                    total_price=total_price,
                    payment_status=payment_status,
                    created_at=current_time.isoformat(),
                    updated_at=(current_time + timedelta(seconds=random.randint(10, 300))).isoformat(),
                    tid=f"T{uuid.uuid4().hex[:16]}"
                )
                events.append(order_event)
                page_tracker.add_page(session_id, order_event["page_url"])
                
                for ordered_item in order_items:
                    if ordered_item in cart_items:
                        cart_items.remove(ordered_item)
                    user_cart_items = [item for item in user_cart_items 
                                    if item["product_id"] != ordered_item["product_id"]]
                
                save_user_order(user_details["user_id"], {
                    "order_id": order_event["event_id"],
                    "total_price": total_price,
                    "order_time": current_time,
                    "status": order_status
                })
                
                current_time += timedelta(seconds=random.randint(300, 600))
                event_count += 1
        
        # 세션 마지막에 로그인 전환 한 번 더 체크 (이벤트 생성 없이)
        check_and_execute_login_conversion()
        
        return events
    
    generated_files = []

    def save_partition_batch(events: List[Dict], batch_num: int):
        if not events:
            return
            
        # 원하는 컬럼 순서 명시적으로 정의
        column_order = [
            'event_id',
            'anonymous_id', 
            'user_id',
            'session_id',
            'event_type',
            'utm_params',
            'page_url',
            'page_view',
            'page_referrer',
            'landing_url',
            'event_timestamp',
            'ingest_timestamp',
            'event_properties'
        ]
        
        df = pd.DataFrame(events)
        
        # 원하는 순서로 컬럼 재정렬
        df = df[column_order]
        
        df['event_properties'] = df['event_properties'].apply(
            lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, dict) else x
        )
        
        output_dir = "data/generated_event_logs"
        os.makedirs(output_dir, exist_ok=True)
        
        filename = f"{output_dir}/partition_{partition_id}_batch_{batch_num}.parquet"
        df.to_parquet(filename, engine='pyarrow', compression='snappy', index=False)
        
        generated_files.append(filename)
        print(f"배치 저장 완료: {filename} ({len(events):,}개 이벤트)")
    
    
    def generate_refund_events() -> List[Dict]:
        refund_events = []
        
        if not hasattr(save_user_order, 'user_orders'):
            return refund_events
        
        for user_id, orders in save_user_order.user_orders.items():
            for order in orders:
                order_time = order['order_time']
                refund_deadline = order_time + timedelta(days=7)
                
                if random.random() < 0.05:
                    refund_delay = random.randint(3600, 604800)
                    refund_time = order_time + timedelta(seconds=refund_delay)
                    
                    if refund_time <= refund_deadline:
                        refund_event = create_event(
                            "refund", 
                            {"user_id": user_id, "anonymous_id": str(uuid.uuid4())}, 
                            str(uuid.uuid4()), 
                            refund_time,
                            order_id=order['order_id'],
                            total_price=order['total_price']
                        )
                        refund_events.append(refund_event)
        
        return refund_events
    
    # 메인 로직
    batch_count = 0
    session_count = 0
    
    for day_offset in range(start_day, end_day):
        current_date = event_start_date + timedelta(days=day_offset)
        current_phase = get_phase_from_day(day_offset)
        daily_dau = calculate_daily_dau(day_offset)
        
        total_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
        print(f"   총 {total_chunks}개 청크로 분할 처리")
        
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            
            # 첫 번째 청크는 헤더 포함, 나머지는 헤더 없이
            mode = 'w' if first_chunk else 'a'
            header = first_chunk
            
            chunk.to_csv(csv_file, mode=mode, header=header, index=False)
            
            current_chunk = (i // chunk_size) + 1
            print(f"   처리 완료: {current_chunk}/{total_chunks} 청크")
            
            partition_events.extend(session_events)
            session_count += 1
            
            if len(partition_events) >= batch_size:
                save_partition_batch(partition_events, batch_count)
                partition_events = []
                batch_count += 1
                gc.collect()
    
    # 환불 이벤트 추가
    refund_events = generate_refund_events()
    partition_events.extend(refund_events)
    
    if partition_events:
        save_partition_batch(partition_events, batch_count)
    
    # 파티션별 커버리지 보고
    all_expected_categories = set()
    for main_cat, sub_cats in category_ranking_dict.items():
        for code in sub_cats.values():
            all_expected_categories.add(code)
    
    coverage_rate = len(used_categories) / len(all_expected_categories) * 100
    print(f"파티션 {partition_id} 완료 - 카테고리 커버리지: {coverage_rate:.1f}% ({len(used_categories)}/{len(all_expected_categories)})")
    
    # return [f"partition_{partition_id}.parquet"]
    return generated_files

In [11]:
def diagnose_partition_files(parquet_files):
    """파티션 파일들의 상태 진단"""
    import pandas as pd
    import os
    
    print("🔍 파티션 파일 진단")
    print("="*60)
    
    for i, file_path in enumerate(parquet_files):
        print(f"\n📄 파일 {i}: {os.path.basename(file_path)}")
        
        # 파일 존재 여부
        if not os.path.exists(file_path):
            print(f"   ❌ 파일이 존재하지 않음")
            continue
            
        # 파일 크기
        size_bytes = os.path.getsize(file_path)
        size_mb = size_bytes / (1024 * 1024)
        print(f"   📊 크기: {size_mb:.2f} MB ({size_bytes:,} bytes)")
        
        if size_bytes == 0:
            print(f"   ❌ 빈 파일")
            continue
        
        # Pandas로 읽기 시도
        try:
            df = pd.read_parquet(file_path)
            print(f"   ✅ 로드 성공: {len(df):,}개 행, {len(df.columns)}개 열")
            print(f"   📋 열 이름: {list(df.columns)}")
            
            # 데이터 타입 확인
            print(f"   🔤 데이터 타입:")
            for col, dtype in df.dtypes.items():
                print(f"      {col}: {dtype}")
                
        except Exception as e:
            print(f"   ❌ 로드 실패: {e}")

# 파일 진단 실행
diagnose_partition_files(partition_files)

수정된 이벤트 로그 생성기 실행
사용 가능한 성별 값: ['U', 'F', 'M']
전체 카테고리 코드 수: 46
-- 모든 카테고리 보장: 46개 제품 생성

--------------------------------------------------
제품 데이터의 최종 메인 카테고리 분포
--------------------------------------------------
  - 상의: 5784개 (41.9%)
  - 바지: 3057개 (22.2%)
  - 아우터: 4078개 (29.6%)
  - 신발: 879개 (6.4%)

--------------------------------------------------
제품 성별 분포
--------------------------------------------------
  - F (여성): 4606개 (33.4%)
  - M (남성): 5713개 (41.4%)
  - U (유니섹스): 3479개 (25.2%)

-- 보장된 카테고리 수: 46개
보장된 카테고리 (처음 10개):
  - 상의>맨투맨/스웨트: 1개
  - 상의>후드 티셔츠: 1개
  - 상의>셔츠/블라우스: 1개
  - 상의>긴소매 티셔츠: 1개
  - 상의>반소매 티셔츠: 1개
  - 상의>피케/카라 티셔츠: 1개
  - 상의>니트/스웨터: 1개
  - 상의>민소매 티셔츠: 1개
  - 상의>기타 상의: 1개
  - 바지>데님 팬츠: 1개

--------------------------------------------------
계절별 제품 분포
--------------------------------------------------

Spring (3450개 제품):
    상의: 1329개 (38.5%)
    바지: 1063개 (30.8%)
    아우터: 882개 (25.6%)
    신발: 176개 (5.1%)

Summer (3450개 제품):
    상의: 1887개 (54.7%)
    바지: 1045개 (30.3%)

In [14]:
def load_all_partitions_dask(parquet_files):
    """Dask로 모든 파티션 로드"""
    import dask.dataframe as dd
    
    print(f"📊 Dask로 {len(parquet_files)}개 파일 로드 중...")
    
    try:
        # Dask DataFrame 생성
        ddf = dd.read_parquet(parquet_files)
        
        print(f"- Dask DataFrame 생성 완료")
        print(f"   파티션 수: {ddf.npartitions}")
        print(f"   열 수: {len(ddf.columns)}")
        print(f"   열 이름: {list(ddf.columns)}")
        
        # 전체 행 수 계산 (시간이 걸릴 수 있음)
        print("- 전체 행 수 계산 중... (시간이 걸릴 수 있습니다)")
        total_rows = len(ddf)
        print(f"   총 행 수: {total_rows:,}개")
        
        return ddf
    except Exception as e:
        print(f"- Dask 로드 실패: {e}")
        return None
    
ddf = load_all_partitions_dask(partition_files)

📊 Dask로 8개 파일 로드 중...
- Dask DataFrame 생성 완료
   파티션 수: 8
   열 수: 13
   열 이름: ['event_id', 'anonymous_id', 'user_id', 'session_id', 'event_type', 'utm_params', 'page_url', 'page_view', 'page_referrer', 'landing_url', 'event_timestamp', 'ingest_timestamp', 'event_properties']
- 전체 행 수 계산 중... (시간이 걸릴 수 있습니다)
   총 행 수: 34,532개


In [15]:
ddf.head(3)

,event_id,anonymous_id,user_id,session_id,event_type,utm_params,page_url,page_view,page_referrer,landing_url,event_timestamp,ingest_timestamp,event_properties
0,165c0fbc-5833-492a-afaa-97c9a315bd83,5b1fa7d1-bf75-426a-90a7-0b9fedad5420,<NA>,979a4fc8-3be0-4185-b84f-aea2d56c1295,view_main,None,https://high-five.store/,<NA>,<NA>,https://high-five.store/products/trending,2025-06-01T03:56:40+00:00,2025-06-01T03:56:43+00:00,<NA>
1,b3dc7c8c-46db-41ea-9c98-28cb6ba27eb5,5b1fa7d1-bf75-426a-90a7-0b9fedad5420,<NA>,979a4fc8-3be0-4185-b84f-aea2d56c1295,promotion_view,None,https://high-five.store/,하이파이브,https://high-five.store/,https://high-five.store/products/trending,2025-06-01T03:56:44+00:00,2025-06-01T03:56:47+00:00,"{""promotion_id"": 454, ""location"": ""main_triple..."
2,d7e75725-3f01-4e2a-8aca-d6558ee9975d,5b1fa7d1-bf75-426a-90a7-0b9fedad5420,<NA>,979a4fc8-3be0-4185-b84f-aea2d56c1295,promotion_view,None,https://high-five.store/,하이파이브,https://high-five.store/,https://high-five.store/products/trending,2025-06-01T03:56:44.409000+00:00,2025-06-01T03:56:49.409000+00:00,"{""promotion_id"": 19, ""location"": ""main_triple_..."


In [16]:
def final_diagnosis():
    """최종 상태 진단 - 정확한 결론 제공"""

    print("🎯 최종 진단 결과")
    print("="*80)
    
    # 파일 로드
    data_dir = "data/generated_event_logs"
    files = [f for f in os.listdir(data_dir) if f.endswith('.parquet')]
    
    dfs = []
    for file in files:
        df = pd.read_parquet(os.path.join(data_dir, file))
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    df['parsed_properties'] = df['event_properties'].apply(
        lambda x: json.loads(x) if pd.notna(x) and x != '' else {}
    )
    
    # 각 문제별 해결 상태 체크
    problems_solved = []
    
    # 1. section_id 불일치 체크
    promo_clicks = df[df['event_type'] == 'promotion_click']
    mismatch_count = 0
    total_click_sessions = 0
    
    if len(promo_clicks) > 0:
        session_section_analysis = {}
        
        # promotion_view section_id 수집
        promo_views = df[df['event_type'] == 'promotion_view']
        for _, row in promo_views.iterrows():
            session_id = row['session_id']
            section_id = row['parsed_properties'].get('section_id')
            if section_id:
                if session_id not in session_section_analysis:
                    session_section_analysis[session_id] = {'view_sections': set(), 'click_sections': set()}
                session_section_analysis[session_id]['view_sections'].add(section_id)
        
        # promotion_click section_id 수집
        for _, row in promo_clicks.iterrows():
            session_id = row['session_id']
            section_id = row['parsed_properties'].get('section_id')
            if section_id:
                if session_id not in session_section_analysis:
                    session_section_analysis[session_id] = {'view_sections': set(), 'click_sections': set()}
                session_section_analysis[session_id]['click_sections'].add(section_id)
        
        # 불일치 계산
        for session_id, data in session_section_analysis.items():
            if data['click_sections']:
                total_click_sessions += 1
                if not data['click_sections'].issubset(data['view_sections']):
                    mismatch_count += 1
    
    section_id_solved = mismatch_count == 0
    problems_solved.append(("section_id 불일치", section_id_solved, f"{mismatch_count}/{total_click_sessions}"))
    
    # 2. promotion_view 수량 체크 (활성화된 프로모션만 고려)
    view_main_count = len(df[df['event_type'] == 'view_main'])
    promotion_view_count = len(df[df['event_type'] == 'promotion_view'])
    
    # 활성화된 프로모션 기반 예상 수량 계산 (완벽히 3개가 아닐 수 있음)
    if view_main_count > 0:
        avg_promo_per_session = promotion_view_count / view_main_count
        promotion_solved = avg_promo_per_session >= 1.0  # 세션당 평균 1개 이상이면 OK
        problems_solved.append(("promotion_view 생성", promotion_solved, f"세션당 평균 {avg_promo_per_session:.2f}개 프로모션"))
    else:
        problems_solved.append(("promotion_view 생성", False, "view_main 이벤트 없음"))
    
    # 3. 로그인 전환 체크 (user_id 변화 기반)
    conversion_sessions = 0
    total_sessions = df['session_id'].nunique()
    
    print("🔍 로그인 전환 분석 중...")
    
    for session_id in df['session_id'].unique():
        session_events = df[df['session_id'] == session_id].sort_values('event_timestamp')
        if len(session_events) == 0:
            continue
            
        sample_name = sample_info[ratio]['name']
        sample_desc = sample_info[ratio]['description']
        
        print(f"\n🔄 {sample_desc} 생성 중...")
        
        # 샘플링 (시간순 유지를 위해 정렬 후 균등 샘플링)
        df_sorted = original_df.sort_values('timestamp').reset_index(drop=True)
        
        if conversion_detected:
            conversion_sessions += 1
    
    # login event_type은 더 이상 존재하지 않아야 함
    login_events = len(df[df['event_type'] == 'login'])
    
    conversion_solved = conversion_sessions > 0 and login_events == 0
    problems_solved.append(("로그인 전환 (user_id 변화)", conversion_solved, f"{conversion_sessions}개 전환 세션, {login_events}개 login 이벤트"))
    
    # 4. 초기 로그인 비율 체크
    initial_logged_sessions = 0
    
    for session_id in df['session_id'].unique():
        session_events = df[df['session_id'] == session_id].sort_values('event_timestamp')
        if len(session_events) == 0:
            continue
            
        first_user_id = session_events.iloc[0]['user_id']
        
        if not (pd.isna(first_user_id) or first_user_id is None):
            initial_logged_sessions += 1
    
    initial_login_rate = initial_logged_sessions / total_sessions * 100 if total_sessions > 0 else 0
    login_ratio_solved = 25 <= initial_login_rate <= 35  # 25-35% 범위면 해결
    problems_solved.append(("초기 로그인 비율", login_ratio_solved, f"{initial_login_rate:.1f}% (목표: 30%)"))
    
    # 5. login event_type 존재 여부 (없어야 함)
    no_login_events = login_events == 0
    problems_solved.append(("login 이벤트 제거", no_login_events, f"{login_events}개 login 이벤트 (0개여야 함)"))
    
    # 6. 전체 데이터 품질 체크
    unique_event_types = df['event_type'].unique()
    expected_event_types = ['view_main', 'promotion_view', 'promotion_click', 'search', 
                           'product_view', 'product_like', 'add_to_cart', 'purchase', 
                           'order', 'brand_like', 'change_cart_item_quantity', 'remove_from_cart',
                           'refund'] 
    
    unexpected_events = [et for et in unique_event_types if et not in expected_event_types]
    data_quality_solved = len(unexpected_events) == 0
    problems_solved.append(("예상치 못한 이벤트 타입", data_quality_solved, f"예상외 이벤트: {unexpected_events if unexpected_events else '없음'}"))
    
    # 결과 출력
    print("📊 각 문제별 해결 상태:")
    print("-" * 60)
    
    solved_count = 0
    for problem, solved, detail in problems_solved:
        status = "✅ 해결됨" if solved else "❌ 미해결"
        print(f"{status} {problem}: {detail}")
        if solved:
            solved_count += 1
    
    # 추가 통계 정보
    print(f"\n📈 데이터 요약:")
    print(f"   - 총 이벤트 수: {len(df):,}개")
    print(f"   - 총 세션 수: {total_sessions:,}개")
    print(f"   - 전환 세션 수: {conversion_sessions:,}개")
    print(f"   - 초기 로그인 세션: {initial_logged_sessions:,}개 ({initial_login_rate:.1f}%)")
    print(f"   - 이벤트 타입 종류: {len(unique_event_types)}개")
    
    print(f"\n🎯 전체 해결 현황: {solved_count}/{len(problems_solved)}개 문제 해결")
    
    if solved_count == len(problems_solved):
        print("🎉 모든 문제가 해결되었습니다!")
        print("✨ 데이터 생성 로직이 완벽하게 작동하고 있습니다.")
    elif solved_count >= len(problems_solved) * 0.75:
        print("👍 대부분의 문제가 해결되었습니다!")
        print("🔧 남은 문제들을 미세 조정하면 완성됩니다.")
    else:
        print("⚠️  더 많은 수정이 필요합니다.")
    
    return solved_count, len(problems_solved)

# 최종 진단 실행
solved, total = final_diagnosis()

🎯 최종 진단 결과
🔍 로그인 전환 분석 중...
📊 각 문제별 해결 상태:
------------------------------------------------------------
✅ 해결됨 section_id 불일치: 0/319
✅ 해결됨 promotion_view 생성: 세션당 평균 2.73개 프로모션
✅ 해결됨 로그인 전환 (user_id 변화): 126개 전환 세션, 0개 login 이벤트
✅ 해결됨 초기 로그인 비율: 31.2% (목표: 30%)
✅ 해결됨 login 이벤트 제거: 0개 login 이벤트 (0개여야 함)
✅ 해결됨 예상치 못한 이벤트 타입: 예상외 이벤트: 없음

📦 1/10 샘플 (10%):
   📊 이벤트 수: 1,003,116개
   📁 Parquet: kafka_sample_1_10.parquet
   📄 JSONL: kafka_sample_1_10.jsonl

🎯 전체 해결 현황: 6/6개 문제 해결
🎉 모든 문제가 해결되었습니다!
✨ 데이터 생성 로직이 완벽하게 작동하고 있습니다.


In [17]:
def analyze_promotion_view_gap():
    """promotion_view 누락 원인 분석"""
    import pandas as pd
    import json
    import os
    
    print("🔍 promotion_view 누락 원인 분석")
    print("="*80)
    
    # 파일 로드
    data_dir = "data/generated_event_logs"
    files = [f for f in os.listdir(data_dir) if f.endswith('.parquet')]
    
    dfs = []
    for file in files:
        df = pd.read_parquet(os.path.join(data_dir, file))
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    df['parsed_properties'] = df['event_properties'].apply(
        lambda x: json.loads(x) if pd.notna(x) and x != '' else {}
    )
    
    # 세션별 분석
    sessions_with_main = set(df[df['event_type'] == 'view_main']['session_id'])
    sessions_with_promo = set(df[df['event_type'] == 'promotion_view']['session_id'])
    
    print(f"📊 view_main이 있는 세션: {len(sessions_with_main)}개")
    print(f"📊 promotion_view가 있는 세션: {len(sessions_with_promo)}개")
    
    # promotion_view가 없는 view_main 세션들
    missing_promo_sessions = sessions_with_main - sessions_with_promo
    print(f"❌ promotion_view가 누락된 세션: {len(missing_promo_sessions)}개")
    
    if missing_promo_sessions:
        print("\n누락된 세션 예시들:")
        for i, session_id in enumerate(list(missing_promo_sessions)[:5]):
            session_events = df[df['session_id'] == session_id].sort_values('event_timestamp')
            event_types = session_events['event_type'].tolist()
            print(f"  {i+1}. 세션 {session_id[:8]}... : {event_types[:10]}")
    
    # promotion_view가 3개 미만인 세션들
    print("\n📊 promotion_view 개수별 세션 분석:")
    promo_count_by_session = df[df['event_type'] == 'promotion_view'].groupby('session_id').size()
    
    for count in [0, 1, 2, 3, 4, 5]:
        if count == 0:
            session_count = len(sessions_with_main) - len(sessions_with_promo)
        else:
            session_count = sum(promo_count_by_session == count)
        print(f"  promotion_view {count}개: {session_count}개 세션")
    
    # 정확히 3개인 세션 비율
    exactly_three = sum(promo_count_by_session == 3)
    print(f"\n🎯 정확히 3개인 세션: {exactly_three}/{len(sessions_with_main)} = {exactly_three/len(sessions_with_main)*100:.1f}%")
    
    return len(missing_promo_sessions), exactly_three

# 분석 실행
missing_count, correct_count = analyze_promotion_view_gap()

🔍 promotion_view 누락 원인 분석
📊 view_main이 있는 세션: 1578개
📊 promotion_view가 있는 세션: 1578개
❌ promotion_view가 누락된 세션: 0개

📊 promotion_view 개수별 세션 분석:
  promotion_view 0개: 0개 세션
  promotion_view 1개: 0개 세션
  promotion_view 2개: 430개 세션
  promotion_view 3개: 1148개 세션
  promotion_view 4개: 0개 세션
  promotion_view 5개: 0개 세션

🎯 정확히 3개인 세션: 1148/1578 = 72.8%


In [18]:
# # 1. 기존 파일 삭제
# import shutil
# import os

# output_dir = "data/generated_event_logs"
# if os.path.exists(output_dir):
#     shutil.rmtree(output_dir)
#     print("기존 로그 파일 삭제 완료")

# # 2. Jupyter 커널 재시작 (수동으로 해야 함)
# print("⚠️  지금 Jupyter 커널을 재시작해주세요!")
# print("   메뉴: Kernel → Restart")

In [19]:
def analyze_generated_event_logs_with_promotion_check(log_directory="data/generated_event_logs"):
    """생성된 이벤트 로그에서 프로모션 이벤트 및 사용자 행동 분석"""
    
    log_files = glob.glob(f"{log_directory}/*.parquet")
    print(f"생성된 파일 수: {len(log_files)}개")
    
    if not log_files:
        print("생성된 파일이 없습니다.")
        return None
    
    try:
        print("Dask DataFrame 로드 중...")
        ddf = dd.read_parquet(f"{log_directory}/*.parquet")
        
        total_events = len(ddf)
        print(f"총 이벤트 수: {total_events:,}개")
        
        # 1. 이벤트 타입별 분포 및 비율 확인
        print("\n=== 이벤트 타입별 분포 및 비율 ===")
        event_type_counts = ddf['event_type'].value_counts().compute()
        for event_type, count in event_type_counts.items():
            percentage = (count / total_events) * 100
            print(f"  {event_type}: {count:,}개 ({percentage:.2f}%)")
        
        # 2. 프로모션 이벤트 상세 분석
        promotion_events = ['promotion_view', 'promotion_click']
        promotion_found = False
        
        print("\n=== 프로모션 이벤트 분석 ===")
        for promo_event in promotion_events:
            if promo_event in event_type_counts.index:
                promotion_found = True
                print(f"\n{promo_event} 이벤트 상세:")
                count = event_type_counts[promo_event]
                percentage = (count / total_events) * 100
                print(f"  총 {promo_event} 이벤트: {count:,}개 ({percentage:.2f}%)")
                
                # 샘플 데이터 확인 - 수정된 부분
                promo_df = ddf[ddf['event_type'] == promo_event]
                # Dask DataFrame에서 샘플 추출
                sample = promo_df.head(3, npartitions=-1)  # npartitions=-1은 모든 파티션에서 검색
                
                # compute()를 사용하여 실제 데이터 가져오기
                try:
                    sample_computed = sample.compute()
                    for idx, row in sample_computed.iterrows():
                        print(f"  샘플 {idx+1}: {row['event_properties']}")
                except Exception as sample_error:
                    print(f"  샘플 데이터 추출 중 오류: {sample_error}")
                    # 대안: 다른 방법으로 샘플 추출
                    try:
                        first_partition = promo_df.get_partition(0).compute()
                        if len(first_partition) > 0:
                            for idx in range(min(3, len(first_partition))):
                                print(f"  샘플 {idx+1}: {first_partition.iloc[idx]['event_properties']}")
                    except Exception as alt_error:
                        print(f"  대안 샘플 추출도 실패: {alt_error}")
        
        if not promotion_found:
            print("\n⚠️  프로모션 이벤트가 생성되지 않았습니다!")
            print("가능한 원인:")
            print("1. promotion_data가 비어있음")
            print("2. 설정된 날짜에 활성 프로모션이 없음")
            print("3. EVENT_CONFIG에서 promotion_click 확률이 0으로 설정됨")
        
        # 3. 사용자 로그인 상태 분석
        print("\n=== 사용자 로그인 상태 분석 ===")
        
        # 전체 세션 분석 (session_id 기준) - Dask에서 compute() 사용
        sessions_df = ddf.groupby('session_id').agg({
            'user_id': ['first', 'last'],
            'anonymous_id': 'first',
            'event_timestamp': ['min', 'max'],
            'event_type': 'count'
        }).compute()
        
        # 컬럼명 정리
        sessions_df.columns = ['first_user_id', 'last_user_id', 'anonymous_id', 'session_start', 'session_end', 'event_count']
        
        total_sessions = len(sessions_df)
        print(f"총 세션 수: {total_sessions:,}개")
        
        # 비로그인으로 시작하는 세션 (first_user_id가 null)
        anonymous_start_sessions = sessions_df[sessions_df['first_user_id'].isna()]
        anonymous_start_count = len(anonymous_start_sessions)
        anonymous_start_percentage = (anonymous_start_count / total_sessions) * 100
        print(f"비로그인으로 시작하는 세션: {anonymous_start_count:,}개 ({anonymous_start_percentage:.2f}%)")
        
        # 비로그인으로 끝나는 세션 (last_user_id가 null)
        anonymous_end_sessions = sessions_df[sessions_df['last_user_id'].isna()]
        anonymous_end_count = len(anonymous_end_sessions)
        anonymous_end_percentage = (anonymous_end_count / total_sessions) * 100
        print(f"비로그인으로 끝나는 세션: {anonymous_end_count:,}개 ({anonymous_end_percentage:.2f}%)")
        
        # 비로그인 → 로그인 전환 세션 (first_user_id는 null, last_user_id는 not null)
        login_conversion_sessions = sessions_df[
            (sessions_df['first_user_id'].isna()) & 
            (sessions_df['last_user_id'].notna())
        ]
        login_conversion_count = len(login_conversion_sessions)
        login_conversion_percentage = (login_conversion_count / anonymous_start_count) * 100 if anonymous_start_count > 0 else 0
        overall_conversion_percentage = (login_conversion_count / total_sessions) * 100
        
        print(f"비로그인 → 로그인 전환 세션: {login_conversion_count:,}개")
        print(f"  - 비로그인 시작 세션 대비: {login_conversion_percentage:.2f}%")
        print(f"  - 전체 세션 대비: {overall_conversion_percentage:.2f}%")
        
        # 처음부터 로그인된 세션
        logged_in_start_sessions = sessions_df[sessions_df['first_user_id'].notna()]
        logged_in_start_count = len(logged_in_start_sessions)
        logged_in_start_percentage = (logged_in_start_count / total_sessions) * 100
        print(f"처음부터 로그인된 세션: {logged_in_start_count:,}개 ({logged_in_start_percentage:.2f}%)")
        
        # 4. 로그인 이벤트 분석
        print("\n=== 로그인 이벤트 분석 ===")
        if 'login' in event_type_counts.index:
            login_event_count = event_type_counts['login']
            login_event_percentage = (login_event_count / total_events) * 100
            print(f"login 이벤트 수: {login_event_count:,}개 ({login_event_percentage:.2f}%)")
            
            # 로그인 이벤트와 전환 세션 수 비교
            print(f"로그인 이벤트 vs 전환 세션 비교:")
            print(f"  - 로그인 이벤트: {login_event_count:,}개")
            print(f"  - 실제 전환 세션: {login_conversion_count:,}개")
            
            if login_event_count != login_conversion_count:
                print(f"  ⚠️  불일치 발견! 차이: {abs(login_event_count - login_conversion_count):,}개")
        else:
            print("⚠️  login 이벤트가 발견되지 않았습니다!")
            if login_conversion_count > 0:
                print("하지만 user_id 변화를 통해 로그인 전환이 감지되었습니다.")
        
        # 5. view_main과 promotion_view 연관성 분석
        print("\n=== view_main과 promotion_view 연관성 분석 ===")
        if 'view_main' in event_type_counts.index and 'promotion_view' in event_type_counts.index:
            view_main_count = event_type_counts['view_main']
            promotion_view_count = event_type_counts['promotion_view']
            
            print(f"view_main 이벤트: {view_main_count:,}개")
            print(f"promotion_view 이벤트: {promotion_view_count:,}개")
            
            # promotion_view는 view_main의 3배 정도 되어야 함 (view_main 후 3개씩 생성)
            expected_promotion_views = view_main_count * 3
            print(f"예상 promotion_view 수 (view_main * 3): {expected_promotion_views:,}개")
            
            if promotion_view_count < expected_promotion_views * 0.8:  # 80% 이하면 경고
                print(f"⚠️  promotion_view 수가 예상보다 적습니다! ({promotion_view_count / expected_promotion_views * 100:.1f}%)")
            else:
                print(f"✅ promotion_view 생성이 정상적입니다 ({promotion_view_count / expected_promotion_views * 100:.1f}%)")
        
        # 6. 세션별 이벤트 분포 통계
        print("\n=== 세션별 이벤트 분포 통계 ===")
        print(f"세션당 평균 이벤트 수: {sessions_df['event_count'].mean():.2f}개")
        print(f"세션당 이벤트 수 중위값: {sessions_df['event_count'].median():.0f}개")
        print(f"세션당 최대 이벤트 수: {sessions_df['event_count'].max():,}개")
        print(f"세션당 최소 이벤트 수: {sessions_df['event_count'].min():,}개")
        
        # 7. 로그인 전환 세션의 샘플 확인
        if login_conversion_count > 0:
            print(f"\n=== 로그인 전환 세션 샘플 (최대 3개) ===")
            sample_conversion_sessions = login_conversion_sessions.head(3)
            
            for idx, (session_id, session_info) in enumerate(sample_conversion_sessions.iterrows()):
                print(f"\n전환 세션 {idx+1}: {session_id}")
                print(f"  - Anonymous ID: {session_info['anonymous_id']}")
                print(f"  - 최종 User ID: {session_info['last_user_id']}")
                print(f"  - 이벤트 수: {session_info['event_count']}개")
                print(f"  - 세션 시간: {session_info['session_start']} ~ {session_info['session_end']}")
                
                # 해당 세션의 이벤트 타입 순서 확인 (Dask에서 compute() 사용)
                session_events = ddf[ddf['session_id'] == session_id].compute()
                session_events = session_events.sort_values('event_timestamp')
                event_sequence = session_events['event_type'].tolist()
                print(f"  - 이벤트 순서: {' → '.join(event_sequence)}")
        
        return ddf
        
    except Exception as e:
        print(f"분석 실패: {e}")
        import traceback
        traceback.print_exc()
        return None

# 실행
check_df = analyze_generated_event_logs_with_promotion_check()

생성된 파일 수: 8개
Dask DataFrame 로드 중...
총 이벤트 수: 34,532개

=== 이벤트 타입별 분포 및 비율 ===
  change_cart_item_quantity: 98개 (0.28%)
  product_like: 1,499개 (4.34%)
  product_view: 23,826개 (69.00%)
  promotion_click: 319개 (0.92%)
  promotion_view: 4,304개 (12.46%)
  add_to_cart: 516개 (1.49%)
  brand_like: 51개 (0.15%)
  search: 2,250개 (6.52%)
  order: 37개 (0.11%)
  remove_from_cart: 54개 (0.16%)
  view_main: 1,578개 (4.57%)

=== 프로모션 이벤트 분석 ===

promotion_view 이벤트 상세:
  총 promotion_view 이벤트: 4,304개 (12.46%)
  샘플 데이터 추출 중 오류: 'DataFrame' object has no attribute 'compute'
  샘플 1: {"promotion_id": 454, "location": "main_triple_banner", "section_id": "454_19", "position": 1}
  샘플 2: {"promotion_id": 19, "location": "main_triple_banner", "section_id": "454_19", "position": 2}
  샘플 3: {"promotion_id": 454, "location": "main_triple_banner", "section_id": "454_19", "position": 1}

promotion_click 이벤트 상세:
  총 promotion_click 이벤트: 319개 (0.92%)
  샘플 데이터 추출 중 오류: 'DataFrame' object has no attribute 'compute'
  샘플 1:

In [20]:
def get_conversion_sessions_list():
    """비로그인→로그인 전환 세션 목록 추출 (user_id 변화 기반)"""
    import pandas as pd
    import json
    import os
    
    print("🔍 비로그인→로그인 전환 세션 목록 추출 (user_id 변화 기반)")
    print("="*80)
    
    # 파일 로드
    data_dir = "data/generated_event_logs"
    files = [f for f in os.listdir(data_dir) if f.endswith('.parquet')]
    
    if not files:
        print("❌ 파일을 찾을 수 없습니다.")
        return []
    
    dfs = []
    for file in files:
        df = pd.read_parquet(os.path.join(data_dir, file))
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    df['parsed_properties'] = df['event_properties'].apply(
        lambda x: json.loads(x) if pd.notna(x) and x != '' else {}
    )
    
    print(f"📊 전체 데이터: {len(df)}개 이벤트, {df['session_id'].nunique()}개 세션")
    
    # login 이벤트 확인 (없어야 함)
    login_events = df[df['event_type'] == 'login']
    print(f"🚫 login 이벤트: {len(login_events)}개 (0개여야 함)")
    
    # 전환 세션 찾기
    conversion_sessions = []
    
    # 각 세션별로 전환 여부 체크
    for session_id in df['session_id'].unique():
        session_events = df[df['session_id'] == session_id].sort_values('event_timestamp')
        
        if len(session_events) == 0:
            continue
        
        # 세션 내에서 user_id 변화 추적
        user_ids = session_events['user_id'].tolist()
        event_types = session_events['event_type'].tolist()
        
        # 전환점 찾기: null/NaN → 실제 값으로 변화하는 지점
        conversion_point = None
        conversion_event_type = None
        conversion_user_id = None
        
        for i in range(len(user_ids)):
            current_id = user_ids[i]
            
            # 현재가 null이 아닌 첫 번째 지점이면서, 이전이 null인 경우
            is_current_not_null = not (pd.isna(current_id) or current_id is None)
            
            if is_current_not_null:
                # 이전 user_id들이 모두 null인지 확인
                prev_ids = user_ids[:i]
                all_prev_null = all(pd.isna(uid) or uid is None for uid in prev_ids)
                
                if all_prev_null and i > 0:  # 첫 번째 이벤트가 아니고, 이전이 모두 null
                    conversion_point = i + 1  # 1부터 시작하는 순서
                    conversion_event_type = event_types[i]
                    conversion_user_id = current_id
                    break
        
        # 전환 조건: null → 값으로 변화한 지점이 있음
        if conversion_point is not None:
            first_user_id = user_ids[0]
            last_user_id = user_ids[-1]
            
            # 추가 검증: 첫 번째는 null, 마지막은 값이 있어야 함
            start_anonymous = pd.isna(first_user_id) or first_user_id is None
            end_logged_in = not (pd.isna(last_user_id) or last_user_id is None)
            
            if start_anonymous and end_logged_in:
                conversion_info = {
                    'session_id': session_id,
                    'conversion_point': conversion_point,
                    'conversion_event_type': conversion_event_type,
                    'conversion_user_id': conversion_user_id,
                    'final_user_id': last_user_id,
                    'total_events': len(session_events),
                    'event_types': event_types,
                    'user_id_sequence': [str(uid) if not (pd.isna(uid) or uid is None) else 'null' for uid in user_ids]
                }
                
                conversion_sessions.append(conversion_info)
    
    print(f"🔄 실제 전환된 세션: {len(conversion_sessions)}개")
    
    # 전환 세션 목록 출력
    print(f"\n📝 전환 세션 상세 목록 ({len(conversion_sessions)}개):")
    print("-" * 130)
    print(f"{'No':<3} {'Session ID':<12} {'전환점':<5} {'전환 이벤트':<12} {'전환 User ID':<12} {'총 이벤트':<6} {'User ID 변화':<20} {'이벤트 순서 (처음 8개)'}")
    print("-" * 130)
    
    for i, session_info in enumerate(conversion_sessions, 1):
        session_id_short = session_info['session_id'][:8] + "..."
        conversion_point = session_info['conversion_point']
        conversion_event = session_info['conversion_event_type']
        conversion_user_short = str(session_info['conversion_user_id'])[:8] + "..." if session_info['conversion_user_id'] else "None"
        total_events = session_info['total_events']
        
        # User ID 변화 표시 (처음 5개)
        user_id_changes = " → ".join(session_info['user_id_sequence'][:5])
        if len(session_info['user_id_sequence']) > 5:
            user_id_changes += "..."
        
        event_sequence = ", ".join(session_info['event_types'][:8])
        if len(session_info['event_types']) > 8:
            event_sequence += "..."
        
        print(f"{i:<3} {session_id_short:<12} {conversion_point:<5} {conversion_event:<12} {conversion_user_short:<12} {total_events:<6} {user_id_changes:<20} {event_sequence}")
    
    # 전환점별 통계
    conversion_points = [s['conversion_point'] for s in conversion_sessions if s['conversion_point']]
    if conversion_points:
        print(f"\n📊 전환점 분포:")
        from collections import Counter
        point_counts = Counter(conversion_points)
        for point, count in sorted(point_counts.items()):
            print(f"   - {point}번째 이벤트에서 전환: {count}개 세션")
    
    # 전환 이벤트 타입별 통계
    conversion_event_types = [s['conversion_event_type'] for s in conversion_sessions if s['conversion_event_type']]
    if conversion_event_types:
        print(f"\n📊 전환 이벤트 타입 분포:")
        from collections import Counter
        event_counts = Counter(conversion_event_types)
        for event_type, count in sorted(event_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"   - {event_type}: {count}개 세션")
    
    # 상세 전환 패턴 분석 (처음 5개 세션)
    print(f"\n🔍 전환 패턴 상세 분석 (처음 5개 세션):")
    print("-" * 100)
    
    for i, session_info in enumerate(conversion_sessions[:5], 1):
        print(f"\n{i}. 세션 {session_info['session_id'][:12]}...")
        print(f"   전환점: {session_info['conversion_point']}번째 이벤트 ({session_info['conversion_event_type']})")
        print(f"   User ID 변화: {' → '.join(session_info['user_id_sequence'])}")
        print(f"   이벤트 순서: {' → '.join(session_info['event_types'])}")
    
    # 전환율 계산
    total_sessions = df['session_id'].nunique()
    conversion_rate = len(conversion_sessions) / total_sessions * 100 if total_sessions > 0 else 0
    
    print(f"\n📈 전환 통계:")
    print(f"   - 전체 세션: {total_sessions:,}개")
    print(f"   - 전환 세션: {len(conversion_sessions):,}개")
    print(f"   - 전환율: {conversion_rate:.2f}%")
    
    return [s['session_id'] for s in conversion_sessions]

# 전환 세션 목록 추출 실행
conversion_session_ids = get_conversion_sessions_list()

print(f"\n📋 전환 세션 ID 목록 ({len(conversion_session_ids)}개):")
print("="*80)
for i in range(0, len(conversion_session_ids), 5):  # 5개씩 묶어서 출력
    batch = conversion_session_ids[i:i+5]
    print(" | ".join([sid[:12] + "..." for sid in batch]))

🔍 비로그인→로그인 전환 세션 목록 추출 (user_id 변화 기반)
📊 전체 데이터: 34532개 이벤트, 1578개 세션
🚫 login 이벤트: 0개 (0개여야 함)
🔄 실제 전환된 세션: 126개

📝 전환 세션 상세 목록 (126개):
----------------------------------------------------------------------------------------------------------------------------------
No  Session ID   전환점   전환 이벤트       전환 User ID   총 이벤트  User ID 변화           이벤트 순서 (처음 8개)
----------------------------------------------------------------------------------------------------------------------------------
1   2393132e...  8     search       ef8004e5...  92     null → null → null → null → null... view_main, promotion_view, promotion_view, promotion_view, promotion_click, product_view, search, search...
2   19e99518...  8     search       01569fc3...  22     null → null → null → null → null... view_main, promotion_view, promotion_view, promotion_view, promotion_click, product_view, search, search...
3   760c6aa0...  8     product_view 71ee09e7...  15     null → null → null → null → null... view_main, promoti

In [37]:
# conversion_session_ids 목록에 있는 session_id만 필터링
filtered_ddf = ddf[ddf['session_id'].isin(conversion_session_ids)]
filtered_ddf.head(10)

,event_id,anonymous_id,user_id,session_id,event_type,utm_params,page_url,page_view,page_referrer,landing_url,event_timestamp,ingest_timestamp,event_properties
32,ab31a84d-61ca-4449-9b4b-207874c08606,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,view_main,None,https://highfive.store/,<NA>,https://highfive.store/,https://highfive.store/,2025-06-01T08:19:38+00:00,2025-06-01T08:19:46+00:00,<NA>
33,b5d3a1f9-42c3-4be5-875e-7b90e6f23610,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,promotion_view,None,https://highfive.store/,하이파이브,https://highfive.store/,https://highfive.store/,2025-06-01T08:19:42+00:00,2025-06-01T08:19:50+00:00,"{""promotion_id"": 454, ""location"": ""main_triple..."
34,d07f71fc-2dee-4841-80a6-cb7bebd078e8,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,promotion_view,None,https://highfive.store/,하이파이브,https://highfive.store/,https://highfive.store/,2025-06-01T08:19:42.341000+00:00,2025-06-01T08:19:49.341000+00:00,"{""promotion_id"": 19, ""location"": ""main_triple_..."
35,77935461-8580-4eb3-8ff6-8c607d8697d2,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,search,None,https://highfive.store/search?q=부츠/워커,부츠/워커 | 하이파이브,https://highfive.store/,https://highfive.store/,2025-06-01T08:19:42.616000+00:00,2025-06-01T08:19:46.616000+00:00,"{""query"": ""부츠/워커"", ""results_count"": 20, ""filte..."
36,aa17ac76-03ac-4f46-ae5e-12691b3436e0,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,search,None,https://highfive.store/search?q=반소매 티셔츠,반소매 티셔츠 | 하이파이브,https://highfive.store/search?q=부츠/워커,https://highfive.store/,2025-06-01T08:20:42.616000+00:00,2025-06-01T08:20:43.616000+00:00,"{""query"": ""반소매 티셔츠"", ""results_count"": 12, ""fil..."
37,fffe2dc4-a973-4a15-9bbc-33a3b5a10532,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,product_view,None,https://highfive.store/products/4057917,하이파이브,https://highfive.store/search?q=반소매 티셔츠,https://highfive.store/,2025-06-01T08:21:14.616000+00:00,2025-06-01T08:21:21.616000+00:00,"{""product_id"": 4057917, ""product_name"": ""센에셜 크..."
38,1bdab1c3-1130-4900-9554-3e87092805c2,4349cc91-2909-4b51-84de-5addf739136b,<NA>,c08c7628-7c28-44a4-b553-e51231db6b78,product_view,None,https://highfive.store/products/3228922,하이파이브,https://highfive.store/products/4057917,https://highfive.store/,2025-06-01T08:23:09.616000+00:00,2025-06-01T08:23:12.616000+00:00,"{""product_id"": 3228922, ""product_name"": ""[TSE]..."
39,92ae8db0-bef7-4c1f-b861-7a69d244360f,4349cc91-2909-4b51-84de-5addf739136b,39af06ed-69f9-4ecc-93b5-5090c7387220,c08c7628-7c28-44a4-b553-e51231db6b78,product_view,None,https://highfive.store/products/4865041,하이파이브,https://highfive.store/products/3228922,https://highfive.store/,2025-06-01T08:25:00.616000+00:00,2025-06-01T08:25:09.616000+00:00,"{""product_id"": 4865041, ""product_name"": ""WERTS..."
40,cb759cbe-c835-460a-8979-c572d7de9125,4349cc91-2909-4b51-84de-5addf739136b,39af06ed-69f9-4ecc-93b5-5090c7387220,c08c7628-7c28-44a4-b553-e51231db6b78,product_view,None,https://highfive.store/products/1558197,하이파이브,https://highfive.store/products/4865041,https://highfive.store/,2025-06-01T08:26:42.616000+00:00,2025-06-01T08:26:45.616000+00:00,"{""product_id"": 1558197, ""product_name"": ""릴스렉드 ..."
41,567e0704-31c4-4bb9-8e84-52c1cfed089e,4349cc91-2909-4b51-84de-5addf739136b,39af06ed-69f9-4ecc-93b5-5090c7387220,c08c7628-7c28-44a4-b553-e51231db6b78,product_view,None,https://highfive.store/products/4554917,하이파이브,https://highfive.store/products/1558197,https://highfive.store/,2025-06-01T08:28:07.616000+00:00,2025-06-01T08:28:15.616000+00:00,"{""product_id"": 4554917, ""product_name"": ""널토 퍼퍼..."


## 4. Kafka로 이벤트 로그 전송

In [38]:
EVENT_TYPE_TO_TOPIC = {
    "view_main": "tracking-view_main-dev",
    "search": "tracking-search-dev",
    "promotion_view": "tracking-promotion_view-dev",
    "promotion_click": "tracking-promotion_click-dev",
    "product_view": "tracking-product_view-dev",
    "add_to_cart": "tracking-add_to_cart-dev",
    "change_cart_item_quantity": "tracking-change_cart_item_quantity-dev",
    "remove_from_cart": "tracking-remove_from_cart-dev",
    "order": "tracking-order-dev",
    "refund": "tracking-refund-dev",
    "product_like": "tracking-product_like-dev",
    "brand_like": "tracking-brand_like-dev"
}

In [53]:
ddf.head(10)

,event_id,anonymous_id,user_id,session_id,event_type,utm_params,page_url,page_view,page_referrer,landing_url,event_timestamp,ingest_timestamp,event_properties
0,b02788b9-2cdf-4849-8701-52db618d09f2,d537af07-fc1f-40d8-8d4a-cec62ba48bc4,<NA>,8344d08b-a870-4595-8577-ffcf1fb20414,view_main,None,https://highfive.store/,<NA>,<NA>,https://highfive.store/categories/outerwear,2025-06-01T01:37:13+00:00,2025-06-01T01:37:22+00:00,<NA>
1,aaf425d4-214d-4405-840c-935cb08fc66d,d537af07-fc1f-40d8-8d4a-cec62ba48bc4,<NA>,8344d08b-a870-4595-8577-ffcf1fb20414,promotion_view,None,https://highfive.store/,하이파이브,https://highfive.store/,https://highfive.store/categories/outerwear,2025-06-01T01:37:16+00:00,2025-06-01T01:37:25+00:00,"{""promotion_id"": 454, ""location"": ""main_triple..."
2,d39b58f5-ad82-496d-b65e-fecc8fd72f75,d537af07-fc1f-40d8-8d4a-cec62ba48bc4,<NA>,8344d08b-a870-4595-8577-ffcf1fb20414,promotion_view,None,https://highfive.store/,하이파이브,https://highfive.store/,https://highfive.store/categories/outerwear,2025-06-01T01:37:16.295000+00:00,2025-06-01T01:37:19.295000+00:00,"{""promotion_id"": 19, ""location"": ""main_triple_..."
3,48eb71d4-ed16-4d2d-8b40-c7fbeca37172,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,view_main,None,https://highfive.store/,<NA>,https://highfive.store/,https://highfive.store/promotions/sale,2025-06-01T06:41:31+00:00,2025-06-01T06:41:35+00:00,<NA>
4,8ae245cb-d862-401d-8e67-5686330d5793,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,promotion_view,None,https://highfive.store/,하이파이브,https://highfive.store/,https://highfive.store/promotions/sale,2025-06-01T06:41:33+00:00,2025-06-01T06:41:38+00:00,"{""promotion_id"": 454, ""location"": ""main_triple..."
5,a9dfb361-bc9a-4880-9379-c9dffc6ec510,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,promotion_view,None,https://highfive.store/,하이파이브,https://highfive.store/,https://highfive.store/promotions/sale,2025-06-01T06:41:33.365000+00:00,2025-06-01T06:41:37.365000+00:00,"{""promotion_id"": 19, ""location"": ""main_triple_..."
6,6ee655de-23d8-4efb-b9f5-193eef918a09,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,search,None,https://highfive.store/search?q=슈트 팬츠/슬랙스,슈트 팬츠/슬랙스 | 하이파이브,https://highfive.store/,https://highfive.store/promotions/sale,2025-06-01T06:41:33.548000+00:00,2025-06-01T06:41:36.548000+00:00,"{""query"": ""슈트 팬츠/슬랙스"", ""results_count"": 45, ""f..."
7,bf73e119-73e0-444b-ac3a-9e6f540f1021,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,product_view,None,https://highfive.store/products/1627892,하이파이브,https://highfive.store/search?q=슈트 팬츠/슬랙스,https://highfive.store/promotions/sale,2025-06-01T06:42:00.548000+00:00,2025-06-01T06:42:07.548000+00:00,"{""product_id"": 1627892, ""product_name"": ""레리플카 ..."
8,2326c088-a3d1-4224-aacf-abccadef2a46,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,product_view,None,https://highfive.store/products/2024427,하이파이브,https://highfive.store/products/1627892,https://highfive.store/promotions/sale,2025-06-01T06:43:06.548000+00:00,2025-06-01T06:43:13.548000+00:00,"{""product_id"": 2024427, ""product_name"": ""라웨트이이..."
9,c304d256-d42c-45ed-a0f8-1e6d503cada9,4ed7186f-2661-42ba-b38f-03638d76c362,7b4038ae-c51e-4b90-8b55-a4ba32813067,f4d97ed4-5490-4b2c-bd4a-88d2ca6df140,product_view,None,https://highfive.store/products/4729424,하이파이브,https://highfive.store/products/2024427,https://highfive.store/promotions/sale,2025-06-01T06:43:23.548000+00:00,2025-06-01T06:43:24.548000+00:00,"{""product_id"": 4729424, ""product_name"": ""TORAI..."


In [60]:
row_3 = ddf.head(10).iloc[7]  # head(4)로 4개 가져온 후 3번째 인덱스
row_3_dict = row_3.to_dict()

In [61]:
row_3_dict

{'event_id': 'bf73e119-73e0-444b-ac3a-9e6f540f1021',
 'anonymous_id': '4ed7186f-2661-42ba-b38f-03638d76c362',
 'user_id': '7b4038ae-c51e-4b90-8b55-a4ba32813067',
 'session_id': 'f4d97ed4-5490-4b2c-bd4a-88d2ca6df140',
 'event_type': 'product_view',
 'utm_params': None,
 'page_url': 'https://highfive.store/products/1627892',
 'page_view': '하이파이브',
 'page_referrer': 'https://highfive.store/search?q=슈트 팬츠/슬랙스',
 'landing_url': 'https://highfive.store/promotions/sale',
 'event_timestamp': '2025-06-01T06:42:00.548000+00:00',
 'ingest_timestamp': '2025-06-01T06:42:07.548000+00:00',
 'event_properties': '{"product_id": 1627892, "product_name": "레리플카 티퍼그 츠팬 [키카]", "category_code": "001003", "price": 43900, "discounted_price": 27990, "gender": "M"}'}

In [41]:
def optimized_dask_to_kafka(
    ddf,
    kafka_bootstrap_servers='localhost:9092',
    event_type_to_topic=None,
    optimize_partitions=True,
    repartition_size='100MB',
    use_dask_delayed=True,
    compression='gzip'
):
    """
    Dask의 고급 기능을 활용한 최적화된 Kafka 전송
    """
    
    # import json
    # from datetime import datetime
    
    if event_type_to_topic is None:
        event_type_to_topic = EVENT_TYPE_TO_TOPIC
    
    print(f"최적화된 Dask-Kafka 전송 시작...")
    start_time = datetime.now()
    
    # 1. 파티션 최적화
    if optimize_partitions:
        print("파티션 최적화 중...")
        # 현재 파티션 상태 확인
        current_size = ddf.map_partitions(len).sum().compute()
        current_partitions = ddf.npartitions
        
        print(f"현재: {current_partitions}개 파티션, {current_size:,}개 행")
        
        # 파티션 크기 기반 리파티셔닝
        if repartition_size:
            ddf = ddf.repartition(partition_size=repartition_size)
            print(f"리파티셔닝 후: {ddf.npartitions}개 파티션")
    
    # 2. 이벤트 타입별 사전 그룹화 (Dask 레벨에서)
    print("이벤트 타입별 사전 그룹화...")
    
    @delayed
    def create_topic_partitions(partition_df):
        """각 파티션을 토픽별로 사전 분리"""
        topic_data = {}
        
        if len(partition_df) == 0:
            return topic_data
            
        # 이벤트 타입별로 그룹화
        for event_type, group_df in partition_df.groupby('event_type'):
            topic = event_type_to_topic.get(event_type)
            if topic:
                if topic not in topic_data:
                    topic_data[topic] = []
                topic_data[topic].append(group_df)
        
        # 같은 토픽의 데이터 결합
        for topic in topic_data:
            if len(topic_data[topic]) > 1:
                topic_data[topic] = pd.concat(topic_data[topic], ignore_index=True)
            else:
                topic_data[topic] = topic_data[topic][0]
        
        return topic_data
    
    # 3. Dask Delayed를 사용한 비동기 처리
    if use_dask_delayed:
        print("Dask Delayed 패턴으로 비동기 처리...")
        
        # 각 파티션에 대해 delayed 작업 생성
        delayed_tasks = []
        for i in range(ddf.npartitions):
            partition = ddf.get_partition(i)
            delayed_task = create_topic_partitions(partition)
            delayed_tasks.append(delayed_task)
        
        # 모든 delayed 작업을 병렬 실행
        with dask.config.set(scheduler='threads', num_workers=8):
            topic_partitions_list = dask.compute(*delayed_tasks)
        
        # 토픽별 데이터 결합
        consolidated_topics = {}
        for topic_partitions in topic_partitions_list:
            for topic, data in topic_partitions.items():
                if topic not in consolidated_topics:
                    consolidated_topics[topic] = []
                consolidated_topics[topic].append(data)
        
        # 최종 토픽별 DataFrame 생성
        final_topic_data = {}
        for topic, data_list in consolidated_topics.items():
            if data_list:
                final_topic_data[topic] = pd.concat(data_list, ignore_index=True)
    
    else:
        # 일반적인 compute 방식
        print("일반 compute 방식으로 처리...")
        df = ddf.compute()
        final_topic_data = {}
        
        for event_type, group_df in df.groupby('event_type'):
            topic = event_type_to_topic.get(event_type)
            if topic:
                final_topic_data[topic] = group_df
    
    # 4. 토픽별 병렬 Kafka 전송
    @delayed
    def send_topic_data_to_kafka(topic, topic_df):
        """특정 토픽의 데이터를 Kafka로 전송"""
        if len(topic_df) == 0:
            return {'topic': topic, 'sent': 0, 'failed': 0}
        
        producer_config = {
            'bootstrap_servers': kafka_bootstrap_servers,
            'value_serializer': lambda v: json.dumps(v, ensure_ascii=False).encode('utf-8'),
            'key_serializer': lambda k: str(k).encode('utf-8') if k else None,
            'acks': 'all',
            'retries': 3,
            'batch_size': 32768,  # 더 큰 배치 크기
            'linger_ms': 100,
            'buffer_memory': 67108864,  # 64MB 버퍼
            'compression_type': compression,
            'max_in_flight_requests_per_connection': 5
        }
        
        producer = KafkaProducer(**producer_config)
        sent_count = 0
        failed_count = 0
        
        try:
            # 벡터화된 메시지 준비
            messages = []
            for _, row in topic_df.iterrows():
                message_data = row.to_dict()
                
                # event_properties 파싱
                if isinstance(message_data.get('event_properties'), str):
                    try:
                        message_data['event_properties'] = json.loads(message_data['event_properties'])
                    except:
                        pass
                
                messages.append({
                    'topic': topic,
                    'value': message_data,
                    'key': row.get('session_id')
                })
            
            # 배치 전송
            batch_size = 1000
            for i in range(0, len(messages), batch_size):
                batch = messages[i:i+batch_size]
                
                # 비동기 전송
                futures = []
                for msg in batch:
                    future = producer.send(**msg)
                    futures.append(future)
                
                # 배치 결과 확인
                for future in futures:
                    try:
                        future.get(timeout=10)
                        sent_count += 1
                    except:
                        failed_count += 1
                
                # 중간 flush
                if i % (batch_size * 10) == 0:
                    producer.flush(timeout=30)
            
            # 최종 flush
            producer.flush(timeout=60)
            
        except Exception as e:
            print(f"토픽 {topic} 전송 중 오류: {e}")
            failed_count += len(topic_df) - sent_count
            
        finally:
            producer.close()
        
        return {
            'topic': topic,
            'sent': sent_count,
            'failed': failed_count,
            'total_messages': len(topic_df)
        }
    
    # 5. 모든 토픽에 대해 병렬 전송
    print(f"{len(final_topic_data)}개 토픽에 대해 병렬 전송 중...")
    
    send_tasks = []
    for topic, topic_df in final_topic_data.items():
        task = send_topic_data_to_kafka(topic, topic_df)
        send_tasks.append(task)
    
    # 병렬 실행
    with dask.config.set(scheduler='threads', num_workers=len(final_topic_data)):
        results = dask.compute(*send_tasks)
    
    # 6. 결과 집계
    total_sent = sum(r['sent'] for r in results)
    total_failed = sum(r['failed'] for r in results)
    elapsed_time = datetime.now() - start_time
    
    print(f"\n=== 최적화된 전송 완료 ===")
    print(f"실행 시간: {elapsed_time}")
    print(f"전송 성공: {total_sent:,}개")
    print(f"전송 실패: {total_failed:,}개")
    print(f"평균 속도: {total_sent / elapsed_time.total_seconds():.2f} 메시지/초")
    
    print(f"\n토픽별 결과:")
    for result in results:
        print(f"  {result['topic']}: {result['sent']:,}/{result['total_messages']:,} 성공")
    
    return {
        'total_sent': total_sent,
        'total_failed': total_failed,
        'elapsed_time': elapsed_time,
        'topic_results': results
    }

# 실시간 스트리밍을 위한 Dask Streaming 최적화
def dask_streaming_to_kafka(
    ddf,
    kafka_bootstrap_servers='localhost:9092',
    event_type_to_topic=None,
    window_size='1s',  # 1초 윈도우
    stream_for_seconds=60
):
    """
    Dask를 활용한 실시간 스트리밍 시뮬레이션
    """
    # import dask.dataframe as dd
    # from dask import delayed
    # import time
    # import numpy as np
    
    if event_type_to_topic is None:
        event_type_to_topic = EVENT_TYPE_TO_TOPIC
    
    print(f"Dask 스트리밍 시뮬레이션 시작 ({stream_for_seconds}초간)")
    
    # 전체 데이터를 시간순으로 정렬
    df = ddf.sort_values('event_timestamp').compute()
    total_messages = len(df)
    
    # 스트리밍 속도 계산 (전체 데이터를 지정된 시간에 걸쳐 전송)
    messages_per_second = total_messages / stream_for_seconds
    window_messages = int(messages_per_second * 1)  # 1초당 메시지 수
    
    print(f"초당 {messages_per_second:.2f}개 메시지로 스트리밍")
    
    @delayed
    def send_window_batch(window_df, window_num):
        """윈도우 단위로 메시지 전송"""
        if len(window_df) == 0:
            return {'window': window_num, 'sent': 0}
        
        producer_config = {
            'bootstrap_servers': kafka_bootstrap_servers,
            'value_serializer': lambda v: json.dumps(v, ensure_ascii=False).encode('utf-8'),
            'key_serializer': lambda k: str(k).encode('utf-8') if k else None,
            'acks': 1,  # 스트리밍에서는 성능을 위해 acks=1
            'retries': 1,
            'batch_size': 16384,
            'linger_ms': 50,
            'compression_type': 'lz4'  # 빠른 압축
        }
        
        producer = KafkaProducer(**producer_config)
        sent_count = 0
        
        try:
            for _, row in window_df.iterrows():
                event_type = row['event_type']
                topic = event_type_to_topic.get(event_type)
                
                if not topic:
                    continue
                
                message_data = row.to_dict()
                
                # 현재 시간으로 타임스탬프 업데이트
                current_time = datetime.now()
                message_data['event_timestamp'] = current_time.isoformat()
                message_data['stream_window'] = window_num
                
                try:
                    producer.send(
                        topic=topic,
                        value=message_data,
                        key=row.get('session_id')
                    )
                    sent_count += 1
                except Exception as e:
                    print(f"윈도우 {window_num} 메시지 전송 실패: {e}")
            
            producer.flush(timeout=5)
            
        finally:
            producer.close()
        
        return {'window': window_num, 'sent': sent_count}
    
    # 윈도우별로 스트리밍
    start_time = time.time()
    total_sent = 0
    
    for window_num in range(stream_for_seconds):
        window_start = window_num * window_messages
        window_end = min((window_num + 1) * window_messages, total_messages)
        
        if window_start >= total_messages:
            break
        
        window_df = df.iloc[window_start:window_end]
        
        # 윈도우 전송 (비동기)
        window_task = send_window_batch(window_df, window_num)
        
        # 1초 대기 (실시간 시뮬레이션)
        elapsed = time.time() - start_time - window_num
        if elapsed < 1.0:
            time.sleep(1.0 - elapsed)
        
        # 중간 결과 확인 (매 10초마다)
        if window_num % 10 == 0:
            try:
                result = window_task.compute()
                total_sent += result['sent']
                print(f"윈도우 {window_num}: {result['sent']}개 전송 (누적: {total_sent:,}개)")
            except:
                pass
    
    print(f"스트리밍 완료: 총 {total_sent:,}개 메시지 전송")
    return total_sent

# # 사용 예시
# if __name__ == "__main__":
#     # 최적화된 배치 전송
#     optimized_results = optimized_dask_to_kafka(
#         ddf=event_logs_20m,
#         kafka_bootstrap_servers='localhost:9092',
#         event_type_to_topic=EVENT_TYPE_TO_TOPIC,
#         optimize_partitions=True,
#         repartition_size='50MB',
#         use_dask_delayed=True,
#         compression='gzip'
#     )
    
    # 실시간 스트리밍 시뮬레이션
    # streaming_count = dask_streaming_to_kafka(
    #     ddf=event_logs_20m.head(10000),
    #     kafka_bootstrap_servers='localhost:9092',
    #     event_type_to_topic=EVENT_TYPE_TO_TOPIC,
    #     stream_for_seconds=120  # 2분간 스트리밍
    # )

In [42]:
# 실시간 스트리밍 시뮬레이션
streaming_count = dask_streaming_to_kafka(
    ddf=ddf,
    kafka_bootstrap_servers='localhost:9092',
    event_type_to_topic=EVENT_TYPE_TO_TOPIC,
    stream_for_seconds=120  # 2분간 스트리밍
)

Dask 스트리밍 시뮬레이션 시작 (120초간)
초당 278.32개 메시지로 스트리밍
스트리밍 완료: 총 0개 메시지 전송


In [89]:
ddf.head(10)

KeyboardInterrupt: 

### 4-3. 생성한 로그 데이터 ES 업로드

In [51]:
def load_parquet_with_dask(file_pattern_or_list=None, file_list_path="event_logs_file_list.txt"):
    """
    Dask를 사용하여 대용량 Parquet 파일을 효율적으로 로드합니다.
    
    Args:
        file_pattern_or_list: 파일 경로, 와일드카드 패턴 또는 파일 경로 목록 (기본값: None)
        file_list_path: 파일 경로 목록이 저장된 텍스트 파일 경로 (기본값: "event_logs_file_list.txt")
        
    Returns:
        dask.dataframe: 로드된 Dask DataFrame
    """

    # 파일 경로 목록 결정
    actual_parquet_files = []
    
    if file_pattern_or_list:
        if isinstance(file_pattern_or_list, str):
            # 단일 문자열인 경우, 와일드카드로 처리
            if '*' in file_pattern_or_list:
                actual_parquet_files = glob.glob(file_pattern_or_list)
            else:
                # 단일 파일 또는 디렉토리
                if os.path.isdir(file_pattern_or_list):
                    # 디렉토리 내 모든 파일 검색
                    parquet_files = []
                    for root, _, files in os.walk(file_pattern_or_list):
                        for file in files:
                            if file.endswith('.parquet'):
                                parquet_files.append(os.path.join(root, file))
                    actual_parquet_files.extend(parquet_files)
                    print(f"디렉토리 '{file_pattern_or_list}'에서 {len(parquet_files)}개 파일을 찾았습니다")
                else:
                    actual_parquet_files = [file_pattern_or_list]
        else:
            # 리스트인 경우 각 항목별 검사
            for item in file_pattern_or_list:
                if os.path.isdir(item):
                    # 디렉토리인 경우 내부 파일 검색
                    parquet_files = []
                    for root, _, files in os.walk(item):
                        for file in files:
                            if file.endswith('.parquet'):
                                parquet_files.append(os.path.join(root, file))
                    actual_parquet_files.extend(parquet_files)
                    print(f"디렉토리 '{item}'에서 {len(parquet_files)}개 파일을 찾았습니다")
                else:
                    actual_parquet_files.append(item)
    elif os.path.exists(file_list_path):
        # 파일 목록 텍스트 파일에서 읽기
        with open(file_list_path, "r") as f:
            paths = [line.strip() for line in f if line.strip()]
            
        # 각 경로 확인
        for path in paths:
            if os.path.isdir(path):
                # 디렉토리인 경우 내부 파일 검색
                parquet_files = []
                for root, _, files in os.walk(path):
                    for file in files:
                        if file.endswith('.parquet'):
                            parquet_files.append(os.path.join(root, file))
                actual_parquet_files.extend(parquet_files)
                print(f"디렉토리 '{path}'에서 {len(parquet_files)}개 파일을 찾았습니다")
            else:
                actual_parquet_files.append(path)
    else:
        # 기본 디렉토리에서 파일 찾기
        for path in ["final_partitions", "."]:
            # 디렉토리 내 모든 .parquet 파일 찾기
            if os.path.isdir(path):
                parquet_files = []
                for root, _, files in os.walk(path):
                    for file in files:
                        if file.endswith('.parquet'):
                            parquet_files.append(os.path.join(root, file))
                actual_parquet_files.extend(parquet_files)
                print(f"디렉토리 '{path}'에서 {len(parquet_files)}개 파일을 찾았습니다")
    
    # 실제 존재하는 파일만 필터링
    actual_parquet_files = [f for f in actual_parquet_files if os.path.exists(f) and os.path.isfile(f)]
    
    if not actual_parquet_files:
        raise ValueError("처리할 수 있는 Parquet 파일이 없습니다.")
    
    print(f"{len(actual_parquet_files)}개의 Parquet 파일을 로드합니다.")
    
    # Dask DataFrame으로 로드
    ddf = dd.read_parquet(actual_parquet_files)
    
    print(f"Dask DataFrame 로드 완료: {ddf.npartitions}개 파티션, {len(ddf.columns)}개 열")
    return ddf

In [ ]:
# 1. 데이터 존재 여부 확인
print("=== 긴급 진단 ===")
print(f"ddf 타입: {type(ddf)}")
print(f"파티션 수: {ddf.npartitions}")

# 실제 크기 확인
actual_size = ddf.map_partitions(len).sum().compute()
print(f"실제 크기: {actual_size:,}행")

if actual_size > 0:
    # 작은 샘플로 테스트
    small_sample = ddf.head(100)  # 100행만
    
    # 통합 함수로 테스트
    test_result = dask_to_kafka_unified(
        ddf=small_sample,
        kafka_bootstrap_servers='localhost:9092',
        event_type_to_topic=EVENT_TYPE_TO_TOPIC,
        mode='stream',
        stream_rate_per_second=10,  # 천천히
        stream_duration_seconds=10,  # 10초만
        dry_run=True,  # 실제 전송 없이 테스트
        progress_callback=create_progress_callback()
    )
    
    print(f"테스트 결과: {test_result}")
else:
    print("❌ ddf가 비어있습니다. 다른 변수를 사용해보세요.")
    print("예: event_logs_20m 또는 event_logs_20m_2weeks")